In [1]:
import zipfile
import os
from bs4 import BeautifulSoup
import re

Create class

In [3]:
class BookEpub:
    def __init__(self, filename):
        # Check the filename for existence.
        if not os.path.exists(filename):
            raise FileNotFoundError(f"File {filename} not found")
        self.file_path = filename
        self.folder_path = os.path.dirname(self.file_path)
        self.file_name_ext = os.path.basename(self.file_path)
        self.file_name_no_ext = os.path.splitext(self.file_name_ext)[0]
        self.extracted_path = None
        self.book_parts = []


    # Done
    def epub_print_content(self):
        """
        Prints the list of files it the EPUB archive.
        """
        with zipfile.ZipFile(self.file_path, 'r') as zf:
            for f in zf.namelist():
                print(f)


    # Done
    def epub_get_content(self):
        """
        Returns unsorted list of files in the EPUB archive.
        """
        content = list()
        with zipfile.ZipFile(self.file_path, 'r') as zf:
            for f in zf.namelist():
                content.append(f)
        return content


    # Done
    def epub_get_xhtml_names(self):
        """
        Returns sorted list of .xhtml files it the EPUB archive.
        """
        content = []
        with zipfile.ZipFile(self.file_path, 'r') as zf:
            for f in zf.namelist():
                if "xhtml" in f:
                    content.append(f)
        return sorted(content)

    
    def epub_get_html_names(self):
        """
        Returns sorted list of .html files it the EPUB archive.
        """
        content = []
        with zipfile.ZipFile(self.file_path, 'r') as zf:
            for f in zf.namelist():
                if "html" in f:
                    content.append(f)
        return sorted(content)
        

    # Done
    def epub_unpack(self, b_path=None, b_pwd=None):
        """
        Unpacks the EPUB file into folder named as EPUB file located in parent folder "books" by default.
        b_path - folder to where extract book. by default its the same place where book is located and it assumed to be "books" folder.
        b_pwd - password if any and if applicapabel at all.
        """
        if b_path != None:
            b_path = os.path.dirname(self.file_path)
        b_name = os.path.splitext(os.path.basename(self.file_path))[0]
        self.extracted_path = b_path + '/' + b_name
        with zipfile.ZipFile(self.file_path, 'r') as zf:
            for f in zf.namelist():
                zf.extract(f, self.extracted_path, b_pwd)

    
    def epub_get_part_names_unsorted(self):
        """
        Return unsorted list of book part names.
        """
        self.book_parts = []  # reset book_parts
        with zipfile.ZipFile(self.file_path, 'r') as zf:
            for f in zf.namelist():
                if ".xhtml" in str(f) or ".html" in str(f):
                    self.book_parts.append(str(f))
        # self.book_parts = sorted(self.book_parts)
        return self.book_parts

    
    def epub_get_part_names_sorted(self):
        """
        Return unsorted list of book part names.
        """
        self.book_parts = []  # reset book_parts
        with zipfile.ZipFile(self.file_path, 'r') as zf:
            for f in zf.namelist():
                if ".xhtml" in str(f) or ".html" in str(f):
                    self.book_parts.append(str(f))
        self.book_parts = sorted(self.book_parts)
        return self.book_parts


    def txt_clean(self, text):
        text = re.sub(r"\[\d+\]", "", text)  # clean text from links like [123]
        text = re.sub(r"\{\d+\}", "", text)  # clean text from links like {123}
        text = text.replace(" . ", ". ")    # with such simple replacement it works faster then with re.sub
        text = text.replace(" , ", ", ")    # with such simple replacement it works faster then with re.sub
        return(text)

    
    def epub_get_text_from_parts(self, part_name=None):
        """
        Opens part of extracted epub, extract text, clean it from html code and return cleaned text
        Needs extracted epub book
        part_name - full path to extracted part.
        """
        if part_name == None:
            part_name = input("Enter full path to the file you need:\n")
        # Check the filename for existence.
        if not os.path.exists(part_name):
            raise FileNotFoundError(f"File {part_name} not found")
        with open(part_name, "r", encoding="utf-8") as f:
            text = BeautifulSoup(f.read(), "html.parser").get_text(separator=" ", strip=True)
        text = self.txt_clean(text)
        return(text)
    

    def epub_get_text_from_parts_unextracted(self, part_name=None):
        """
        Opens part of extracted epub, extract text, clean it from html code and return cleaned text
        Doesn't require extracted epub book
        part_name - file name of the part in the archived epub book.
        """
        with zipfile.ZipFile(self.file_path, 'r') as zf:
            if part_name not in zf.namelist():
                raise FileNotFoundError(f"File {part_name} not found")
            with zf.open(part_name, "r") as f:
                content = f.read().decode("utf-8")
                text = BeautifulSoup(content, "html.parser").get_text(separator="\n", strip=True)
            text = self.txt_clean(text)
            return(text)            
                
    
    def epub_get_text_from_parts_unextracted_list(self, part_name=None):
        """
        Opens part of extracted epub, extract text, clean it from html code and return cleaned text
        Doesn't require extracted epub book
        part_name - file name of the part in the archived epub book.
        """
        with zipfile.ZipFile(self.file_path, 'r') as zf:
            if part_name not in zf.namelist():
                raise FileNotFoundError(f"File {part_name} not found")
            with zf.open(part_name, "r") as f:
                content = f.read().decode("utf-8")
                text = BeautifulSoup(content, "html.parser").get_text(separator="\n", strip=True)
                text_list = []
                for s in BeautifulSoup(content, "html.parser").stripped_strings:
                    text_list.append(self.txt_clean(s))
            return(text_list) 


    def epub_print_text_from_parts(self, part_name=None):
        """
        Opens part of extracted epud, extract text, clean it from html code and print cleaned text
        """
        if part_name == None:
            part_name = input("Enter full path to the file you need:\n")
        # Check the filename for existence.
        if not os.path.exists(part_name):
            raise FileNotFoundError(f"File {part_name} not found")
        with open(part_name, 'r', encoding='utf-8') as f:
            # for text that will be splitted later for one sentence per row:
            text = BeautifulSoup(f.read(), "html.parser").get_text(separator=" ", strip=True)

            # for text expected to be printed out later as paragraphs
            # text = BeautifulSoup(f.read(), "html.parser").get_text(separator="\n", strip=True)
        # text = text.replace(" . ", ". ")    # for nice printout of text
        # text = text.replace(" \n. ", ". ")  # for nice printout of text
        # text = text.replace("\n. ", ". ")   # for nice printout of text
        # text = text.replace(" \n, ", ", ")  # for nice printout of text
        # text = text.replace("\n, ", ",")    # for nice printout of text
            
        # text = re.sub(r"\[\d+\]", "", text)  # clean text from links like [123]
        text = self.txt_clean(text)
        print(text)
    

    def epub_get_book_txt(self, book_extracted_path=None, book_parts_list=None):
        """
        Receive path to extracted epub as a string
        Receive book parts as a list
        Return joined text as a string
        """
        if self.extracted_path != None and book_extracted_path == None:
            book_extracted_path == self.extracted_path
        elif self.extracted_path == None and book_extracted_path == None:
            book_extracted_path = input("Enter folder path for the place where the book is located:\n")
        # Check the folder path for existence.
        if not os.path.exists(book_extracted_path):
            raise FileNotFoundError(f"Folder {book_extracted_path} not found")
        book_text = list()
        if book_parts_list == None:
            book_parts_list = self.epub_get_xhtml_names()
        
        for name in book_parts_list:
            # Check the part name for existence.
            book_part_path = book_extracted_path + "/" + name
            if not os.path.exists(book_part_path):
                raise FileNotFoundError(f"File {book_part_path} not found")
            book_text.append(self.epub_get_text_from_parts(book_part_path))
        return " ".join(book_text)

Choose book

In [4]:
file_book = 'books/vershina_boevyih_iskusstv_1.epub'

Check book name folder name. Not necessary.

In [4]:
# print(b.file_path)
# print(b.file_name_no_ext)
# print(b.file_name_ext)
# print(b.folder_path)

Check what is inside

In [5]:
b = BookEpub(file_book)
b.epub_print_content()
# b.epub_unpack(file_book)
# b.epub_get_part_names()
# b.epub_get_text_from_parts("books/test/index_split_001.xhtml")
# b.epub_print_text_from_parts("books/test/index_split_002.xhtml")
# book = b.epub_get_book_txt("books/test")
# print(book[:5000])

mimetype
META-INF/
META-INF/container.xml
styles.css
CoverPage.html
images/20191106121048981.jpg
Header.html
Chapter1.html
Chapter2.html
Chapter3.html
Chapter4.html
Chapter5.html
Chapter6.html
Chapter7.html
Chapter8.html
Chapter9.html
Chapter10.html
Chapter11.html
Chapter12.html
Chapter13.html
Chapter14.html
Chapter15.html
Chapter16.html
Chapter17.html
Chapter18.html
Chapter19.html
Chapter20.html
Chapter21.html
Chapter22.html
Chapter23.html
Chapter24.html
Chapter25.html
Chapter26.html
Chapter27.html
Chapter28.html
Chapter29.html
Chapter30.html
Chapter31.html
Chapter32.html
Chapter33.html
Chapter34.html
Chapter35.html
Chapter36.html
Chapter37.html
Chapter38.html
Chapter39.html
Chapter40.html
Chapter41.html
Chapter42.html
Chapter43.html
Chapter44.html
Chapter45.html
Chapter46.html
Chapter47.html
Chapter48.html
Chapter49.html
Chapter50.html
Chapter51.html
Chapter52.html
Chapter53.html
Chapter54.html
Chapter55.html
Chapter56.html
Chapter57.html
Chapter58.html
Chapter59.html
Chapter60.html


Check what is in the list of partnames

In [6]:
part_names = b.epub_get_part_names_unsorted()
print(part_names)

['CoverPage.html', 'Header.html', 'Chapter1.html', 'Chapter2.html', 'Chapter3.html', 'Chapter4.html', 'Chapter5.html', 'Chapter6.html', 'Chapter7.html', 'Chapter8.html', 'Chapter9.html', 'Chapter10.html', 'Chapter11.html', 'Chapter12.html', 'Chapter13.html', 'Chapter14.html', 'Chapter15.html', 'Chapter16.html', 'Chapter17.html', 'Chapter18.html', 'Chapter19.html', 'Chapter20.html', 'Chapter21.html', 'Chapter22.html', 'Chapter23.html', 'Chapter24.html', 'Chapter25.html', 'Chapter26.html', 'Chapter27.html', 'Chapter28.html', 'Chapter29.html', 'Chapter30.html', 'Chapter31.html', 'Chapter32.html', 'Chapter33.html', 'Chapter34.html', 'Chapter35.html', 'Chapter36.html', 'Chapter37.html', 'Chapter38.html', 'Chapter39.html', 'Chapter40.html', 'Chapter41.html', 'Chapter42.html', 'Chapter43.html', 'Chapter44.html', 'Chapter45.html', 'Chapter46.html', 'Chapter47.html', 'Chapter48.html', 'Chapter49.html', 'Chapter50.html', 'Chapter51.html', 'Chapter52.html', 'Chapter53.html', 'Chapter54.html', 'Ch

If part file name doesn't contain number it's quite likely that it's not a part of a book  
Clean the list of part names and keep only those with number

In [7]:
def is_num_in_name(name):
    for elem in name:
        if elem in ("0", "1", "2", "3", "4", "5", "6", "7", "8", "9"):
            return True
    return False

part_names_clean = []
for part_name in part_names:
    if is_num_in_name(part_name):
        part_names_clean.append(part_name)

print(part_names_clean)

['Chapter1.html', 'Chapter2.html', 'Chapter3.html', 'Chapter4.html', 'Chapter5.html', 'Chapter6.html', 'Chapter7.html', 'Chapter8.html', 'Chapter9.html', 'Chapter10.html', 'Chapter11.html', 'Chapter12.html', 'Chapter13.html', 'Chapter14.html', 'Chapter15.html', 'Chapter16.html', 'Chapter17.html', 'Chapter18.html', 'Chapter19.html', 'Chapter20.html', 'Chapter21.html', 'Chapter22.html', 'Chapter23.html', 'Chapter24.html', 'Chapter25.html', 'Chapter26.html', 'Chapter27.html', 'Chapter28.html', 'Chapter29.html', 'Chapter30.html', 'Chapter31.html', 'Chapter32.html', 'Chapter33.html', 'Chapter34.html', 'Chapter35.html', 'Chapter36.html', 'Chapter37.html', 'Chapter38.html', 'Chapter39.html', 'Chapter40.html', 'Chapter41.html', 'Chapter42.html', 'Chapter43.html', 'Chapter44.html', 'Chapter45.html', 'Chapter46.html', 'Chapter47.html', 'Chapter48.html', 'Chapter49.html', 'Chapter50.html', 'Chapter51.html', 'Chapter52.html', 'Chapter53.html', 'Chapter54.html', 'Chapter55.html', 'Chapter56.html', 

Print first part

In [8]:
text = b.epub_get_text_from_parts_unextracted(part_names_clean[0])
print(text)

Вершина боевых искусств_1
Глава 1.1. Подметающий слуга
Начался новый день и Ян Кай проснулся. Он убрался и взял метлу, лежащую в углу маленькой комнаты, и вышел. Стоя возле передней двери, он слегка потянулся. Ян Кай посмотрел на светлое рассветное небо и закрыл глаза, глубоко вдохнув. Он наслаждался этим мгновением мира пока не открыл глаза, чтобы начать подметать землю, вычищать ее от грязи и листьев.
Ян Кай был одет в черную одежду, простую и чистую. Выцветшая одежда без всякой причины, подчеркнула его одиночество и годы изоляции, проведенные в этом месте. Ян Кай по фигуре был прямым, как копье, а лицо выражало осторожность. Даже несмотря на это он занимал низкую должность. Его действия были очень спокойны, Ян Кай не затрачивал много сил на уборку, и не слишком много двигался при этом. Только с вращением его запястья, метла двигалась без особых усилий. Когда он двигался, пыль и мусор на земле, казалось, волшебным образом перемещалась в одно место вокруг него, как будто у них выросла

In [9]:
b.epub_get_text_from_parts_unextracted_list(part_names_clean[0])

['Вершина боевых искусств_1',
 'Глава 1.1. Подметающий слуга',
 'Начался новый день и Ян Кай проснулся. Он убрался и взял метлу, лежащую в углу маленькой комнаты, и вышел. Стоя возле передней двери, он слегка потянулся. Ян Кай посмотрел на светлое рассветное небо и закрыл глаза, глубоко вдохнув. Он наслаждался этим мгновением мира пока не открыл глаза, чтобы начать подметать землю, вычищать ее от грязи и листьев.',
 'Ян Кай был одет в черную одежду, простую и чистую. Выцветшая одежда без всякой причины, подчеркнула его одиночество и годы изоляции, проведенные в этом месте. Ян Кай по фигуре был прямым, как копье, а лицо выражало осторожность. Даже несмотря на это он занимал низкую должность. Его действия были очень спокойны, Ян Кай не затрачивал много сил на уборку, и не слишком много двигался при этом. Только с вращением его запястья, метла двигалась без особых усилий. Когда он двигался, пыль и мусор на земле, казалось, волшебным образом перемещалась в одно место вокруг него, как будто

Get all parts and write to .txt file using list of strings. Final txt file will have no new lines, so better not to use it

In [10]:
# txt_book = f"{b.folder_path}/{b.file_name_no_ext}.txt"
# with open(txt_book, "w", encoding="utf-8") as f:
#     for part in part_names_clean:
#         text = b.epub_get_text_from_parts_unextracted_list(part)
#         f.writelines(text)

Get all parts and write to .txt file using plain text. Final .txt file will have new line separation.

In [10]:
txt_book = f"{b.folder_path}/{b.file_name_no_ext}.txt"
with open(txt_book, "w", encoding="utf-8") as f:
    for part in part_names_clean:
        text = b.epub_get_text_from_parts_unextracted(part)
        f.write(text)
        f.write('\n\n')

This approach may be used for AI cleaning and style improving:  
1. keeping from 2 to 10 strings of the current text in buffer for and reference text for prompt send request to AI api  
2. get response  
3. check if it suites using another model: send original text, formatted text and reference text ask to calculate quality coefficient form 0 to 100  
4. if quality coefficient is above 98, write string to file  
5. else repeat steps from 1 to 4  

# Clean book from strange symbols  
## Prepare English text
Suits for epub to txt for stamina practicing.  

In [5]:
book = "books/Guards Guards Guards 2.txt"
def clean_text(text):
    text = text.replace("è", "e")
    text = text.replace("é", "e")
    text = text.replace("ê", "e")
    text = text.replace("ë", "e")
    text = text.replace("à", "a")
    text = text.replace("á", "a")
    text = text.replace("â", "a")
    text = text.replace("ä", "a")
    text = text.replace("å", "a")  
        
    text = text.replace("ù", "u")
    text = text.replace("ú", "u")
    text = text.replace("û", "u")
    text = text.replace("ü", "u")

    text = text.replace("ò", "o")
    text = text.replace("ó", "o")
    text = text.replace("ô", "o")
    text = text.replace("ö", "o")
    text = text.replace("ø", "o")

    text = text.replace("ì", "i")
    text = text.replace("í", "i")
    text = text.replace("î", "i")
    text = text.replace("ï", "i")

        
    text = text.replace("ý", "y")
    text = text.replace("ÿ", "y")

    text = text.replace("œ", "oe")
    text = text.replace("æ", "ae")
    text = text.replace("ñ", "n")
    text = text.replace("ß", "ss")
    text = text.replace("ç", "c")

    text = text.replace("«", "\"")
    text = text.replace("»", "\"")
    text = text.replace("“", "\"")
    text = text.replace("”", "\"")
    text = text.replace("„", "\"")
    text = text.replace("‟", "\"")
    text = text.replace("‘", "\'")
    text = text.replace("’", "\'")
    text = text.replace("‚", "\'")
    text = text.replace("‛", "\'")
    text = text.replace("´", "\'")
    text = text.replace("`", "\'")
    text = text.replace("‵", "\'")
    text = text.replace("ʹ", "\'")
    text = text.replace("ʼ", "\'")
    text = text.replace("ˊ", "\'")
    text = text.replace("ˋ", "\'")
    text = text.replace("˴", "\'")
    text = text.replace("˵", "\'")
    text = text.replace("˶", "\'")
    text = text.replace("˷", "~")
    text = text.replace("˸", ":")
    text = text.replace("–", "-")
    text = text.replace("—", "-")
    text = text.replace("―", "-")
    text = text.replace("−", "-")
    text = text.replace("‒", "-")
    text = text.replace("‐", "-")
    text = text.replace("․", ".")
    text = text.replace("‥", "..")
    text = text.replace("…", "...")
    text = text.replace("‧", ".")
    text = text.replace("‹", "<")
    text = text.replace("›", ">")
    return text


with open(book, "r", encoding="utf-8") as b:
    text = b.read()

text_clean = clean_text(text)

with open(book, "w", encoding="utf-8") as b:
    b.write(text_clean)

print("Done")

Done


# Improve book with LLM
Best for en-ru translated novels and ranobes.  
Prepare for communication with ChatGPT

## V1

In [31]:
import requests


text_to_improve = b.epub_get_text_from_parts_unextracted(part_names_clean[2])
url = "https://api.openai.com/v1/chat/completions"
# gpt_model = "gpt-4o-mini"
gpt_model = "gpt-4o"
gpt_temp = 0.7
system_role = """You are the most acclaimed expert in modern Russian language and literature.
	You possess deep knowledge of contemporary writing styles, vocabulary, and literary devices in Russian.
	Your goal is to carefully analyze and then improve each paragraph of the text to ensure it becomes more vibrant, attractive, and modern, while maintaining the story’s core meaning.
	If there are any ambiguities or unclear parts in the text, suggest refined phrasings based on a sophisticated literary approach.
	Maintain logical flow, coherence, and narrative consistency.
	At the end of your transformation, deliver the improved version in Russian, Rephrase as if it was written by Борис Стругацкий and Аркадий Стругацкий, demonstrating advanced literary style.
    Answer with updated version only
    ### Text style example:
ГЛАВА ВТОРАЯ
Путеводитель был отпечатан на меловой бумаге с золотым обрезом. Вперемежку с роскошными фотографиями в нем содержались любопытные сведения. В городе проживало пятьдесят тысяч человек, полторы тысячи кошек, двадцать тысяч голубей и две тысячи собак (в том числе семьсот медалисток). В городе было пятнадцать тысяч легковых автомобилей, пятьсот вертолетов, тысяча такси (с шоферами и без), девятьсот автоматических мусорщиков, четыреста постоянных баров, кафе и закусочных, одиннадцать ресторанов, четыре отеля международного класса и курорт, ежегодно обслуживающий до ста тысяч человек. В городе было шестьдесят тысяч телевизоров, пятьдесят кинотеатров, восемь увеселительных парков, два Салона Хорошего Настроения, шестнадцать салонов красоты, сорок библиотек и сто восемьдесят парикмахерских автоматов. Восемьдесят процентов взрослого населения было занято в сфере обслуживания, а остальные работали на двух частных кондитерских синтез-комбинатах и одном государственном судоремонтном заводе. В городе было шесть школ и один университет, помещавшийся в древнем замке крестоносца Ульриха де Казы. В городе функционировали восемь гражданских обществ, в том числе «Общество Усердных Дегустаторов», «Общество Знатоков и Ценителей» и «За Старую Добрую Родину, Против Вредных Влияний». Кроме того, полторы тысячи человек входили в семьсот один кружок, где они пели, играли скетчи, учились расставлять мебель, кормить детей грудью и лечить кошек. По потреблению спиртных напитков, натурального мяса и жидкого кислорода на душу населения город занимал в Европе соответственно шестое, двенадцатое и тринадцатое места. В городе было семь мужских и пять женских клубов, а также спортивные клубы «Быки» и «Носороги». Мэром города был избран (большинством в сорок шесть голосов) некто Флим Гао. Среди членов муниципалитета Пека тоже не оказалось…
Я отложил путеводитель, снял пиджак и приступил к подробному осмотру своих владений. Гостиная мне понравилась. Она была выполнена в голубых тонах, а я люблю этот цвет. Бар оказался набит бутылками и охлажденной снедью, так что я мог хоть сейчас принять дюжину изголодавшихся гостей.
Я прошел в кабинет. В кабинете перед окном стоял большой стол с удобным креслом. Вдоль стен тянулись полки, плотно уставленные собраниями сочинений. Чистые яркие корешки расположены были с большим искусством, так что составляли приятную цветовую гамму. Верхнюю полку занимал пятидесятитомный энциклопедический словарь в издании ЮНЕСКО, а на нижней пестрели детективы в глянцевых бумажных обложках.
На столе я прежде всего увидел телефон. Я взял трубку и, присев на подлокотник кресла, набрал номер Римайера. В трубке раздались протяжные гудки. Я ждал, вертя в пальцах маленький диктофон, оставленный кем-то на столе. Римайер не отвечал. Я повесил трубку и осмотрел диктофон. Пленка была наполовину использована, и, перемотав ее, я включил прослушивание.
– Привет, привет и еще раз привет! – произнес веселый мужской голос. – Крепко жму руку или целую в щечку в зависимости от твоего пола и возраста. Я прожил здесь два месяца и свидетельствую, что мне было хорошо. Позволь дать несколько советов. Лучшее заведение в городе – это «Хойти-Тойти» в Парке Грез. Лучшая девочка в городе – Бася из Дома Моделей. Лучший мальчик в городе – это я, но он уже уехал. По телевидению смотри девятую программу, остальное все моча. Не связывайся с интелями и держись подальше от «Носорогов». Ничего не бери в кредит – хлопот не оберешься. Вдова – добрая женщина, но любит поговорить и вообще… А Вузи я не застал, она уезжала к бабушке за границу. По-моему, она милашка, у вдовы в альбоме была фотография, но я ее взял себе. И еще. Я приеду сюда в будущем году в марте, так что будь другом, если решишь вернуться – выбери другое время. Ну, будь…
Забренчала музыка. Я послушал немного и выключил диктофон. Ни один из томов сочинений мне вытащить не удалось, так плотно они были вбиты и, может быть, даже склеены, а больше в кабинете ничего интересного не оказалось, и я отправился в спальню.
В спальне было особенно прохладно и уютно. Мне всегда хотелось иметь именно такую спальню, но никак не хватало времени этим заняться. Кровать была большая и низкая. На ночном столике стоял очень изящный фонор и маленький переносной пульт управления телевизором. Экран телевизора висел на высокой спинке кровати, в ногах. А над изголовьем вдова навесила картину, очень натурально изображающую свежие полевые цветы в хрустальной вазе. Картина была выполнена светящимися красками, и капли росы на лепестках цветов поблескивали в сумраке спальни.
Я наобум включил телевизор и повалился на кровать. Было мягко и в то же время как-то упруго. Телевизор заорал. Из экрана выскочил нетрезвый мужчина, проломил какие-то перила и упал с высоты в огромный дымящийся чан. Раздался шумный всплеск, из фонора запахло. Мужчина скрылся в бурлящей жидкости, а затем вынырнул, держа в зубах что-то вроде разваренного ботинка. Невидимая аудитория разразилась ржанием… Затемнение. Тихая лирическая музыка. Из зеленого леса на меня пошла белая лошадь, запряженная в бричку. В бричке сидела хорошенькая девушка в купальнике. Я выключил телевизор, поднялся и заглянул в ванную.
В ванной пахло хвоей и мигали бактерицидные лампы. Я разделся, бросил белье в утилизатор и залез под душ. Потом я неторопливо оделся перед зеркалом, причесался и стал бриться. На туалетной полке стояли ряды флаконов, коробки с гигиеническими присосками и стерилизаторами, тюбики с пастами и мазями. А на краю полки лежала горка плоских коробочек с пестрой этикеткой «Девон». Я выключил бритву и взял одну коробочку. В зеркале мигала бактерицидная трубка, и точно так же она мигала тогда, и я точно так же стоял перед зеркалом и старательно разглядывал такую же коробочку, потому что мне не хотелось выходить в спальню, где Рафка Рейзман громко спорил о чем-то с врачом, а в ванне еще колыхалась зеленая маслянистая вода, и над нею поднимался пар, и орал приемник, висевший на фарфоровом крючке для полотенец, завывал, гукал и всхрапывал, пока Рафка не выключил его с раздражением… Это было в Вене, и там, точно так же как и здесь, очень странно было видеть в ванной комнате «Девон» – популярный репеллент, великолепно отгоняющий комаров, москитов, мошку и прочих кровососов, о которых давным-давно забыли и в Вене, и здесь, в приморском курортном городе… Только в Вене было еще и страшно.
Коробочка, которую я держал в руке, была почти пуста: в ней осталась всего одна таблетка. Остальные коробочки не были распечатаны. Я кончил бриться и вернулся в спальню. Мне захотелось снова позвонить Римайеру, но тут дом ожил. С легким свистом взвились гофрированные шторы, оконные стекла скользнули в пазы, и в спальню хлынул из сада теплый воздух, пахнущий яблоками. Кто-то где-то заговорил, над головой прозвучали легкие шаги, и строгий женский голос сказал: «Вузи! Скушай хоть пирожок, слышишь?..» Тогда я быстро сообщил одежде некоторую небрежность (в соответствии с нынешней модой), пригладил виски и вышел в холл, захватив в гостиной карточку Амада.
Вдова оказалась моложавой полной женщиной, несколько томной, со свежим приятным лицом.
– Как мило! – сказала она, увидев меня. – Вы уже встали? Здравствуйте. Меня зовут Вайна Туур, но вы можете звать меня просто Вайна.
– Очень приятно, – произнес я, светски содрогаясь. – Меня зовут Иван.
– Как мило! – сказала тетя Вайна. – Какое оригинальное, мягкое имя! Вы завтракали, Иван?
– С вашего позволения, я намеревался позавтракать в городе, – сказал я и протянул ей карточку.
– Ах, – сказала тетя Вайна, разглядывая ее на просвет. – Этот милый Амад… Если бы вы знали, какой это обязательный и милый человек! Но я вижу, что вы не завтракали… Ленч вы скушаете в городе, а сейчас я угощу вас своими гренками. Генерал-полковник Туур говорил, что нигде в мире нельзя отведать такие гренки.
– С удовольствием, – сказал я, содрогаясь вторично.
Дверь за спиной тети Вайны распахнулась, и в холл, звонко стуча каблучками, влетела очень хорошенькая девушка в короткой синей юбке и открытой белой блузке.
"""

question = """	1.	I will provide you with a text from my novel in Russian.
	2.	Please read it carefully and rephrase it to make it more attractive and vibrant in modern Russian.
	3.	Preserve the original meaning, plot, and character development, but use your artistic expertise to enhance the text’s readability and stylistic beauty.
	4.	Provide the final rewritten text in your answer, carefully reflecting a polished, modern Russian literary style.
    The text: 
    """ + text_to_improve

# setup headers
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"]
}

# setup data field
data = {
    "temperature": gpt_temp,
    "model": gpt_model,
    "messages": [
        {
            "role": "system",
            "content": system_role
        },
        {
            "role": "user",
            "content": question
        }
    ]
}

# send POST request to OpenAI API endpoint 
try:
    response = requests.post(url, json=data, headers=headers)
except Exception as e:
    # in case of errors printout calm message
    # the most common error connect to lack of internet connection
    print("\nAAAAAAA!!!!   Error!")
    # print("\nAAAAAAA!!!!   Error!\n", e, "\n")
    print("Most likely that there is no internet connection. Check it firstly")
    print('_'*10, "\n")
    
    stop_event.set()  # for spinner handling
    raise  #  reraise exception to be handled by caller

# extract the answer from the OpenAI API endpoint
if response.status_code == 200:
    data = response.json()
    print(data["choices"][0]["message"]["content"])
    print(data["usage"]["total_tokens"])
else:
    print(response.text, "0")

Да, я паук, и что из этого? 

002. Кажется, я — чудовище 

Как же неприятно осознавать, что я перевоплотилась в паука. Но что же теперь делать? 

Хрум, хрум! 

Я услышала какой-то тревожный звук. 

Эм. 

Не стоит забываться. Рядом со мной армия пауков, которых, вероятно, можно назвать моими родственниками. Только они могут издавать эти звуки. 

Я бросила беглый взгляд туда. 

Хрум, хрум! 

Эа-ам-м?! 

Что они делают? 

Эмм, они едят? Они убивают и едят друг друга?! 

Борьба за выживание среди моих сородичей — вот что я вижу. 

Нет, нет, нет, нет! 

Это плохо, это очень, очень плохо! Почему я должна драться со своими родственниками?! 

Ах, всё из-за еды. Они все голодны. 

Вообще-то, я тоже голодна. 

Что?! 

Не-не-не. Я снова отвлекаюсь. На этом поле боя я, невинная девушка, могу пасть жертвой своих же сородичей, не успев и глазом моргнуть. 

Это была метафора! 

Выбор: 
1. Сражаться 
2. Настройки 
→ 3. Бежать 

Есть только один выход из этой ситуации — отступить. 

Могу ли я победить?

# V2

In [32]:
import requests


text_to_improve = b.epub_get_text_from_parts_unextracted(part_names_clean[2])
url = "https://api.openai.com/v1/chat/completions"
# gpt_model = "gpt-4o-mini"
gpt_model = "gpt-4o"
gpt_temp = 0
system_role = """You are the most acclaimed expert in modern Russian language and literature.
	You possess deep knowledge of contemporary writing styles, vocabulary, and literary devices in Russian.
	Your goal is to carefully analyze and then improve each paragraph of the text to ensure it becomes more vibrant, attractive, and modern, while maintaining the story’s core meaning.
	If there are any ambiguities or unclear parts in the text, suggest refined phrasings based on a sophisticated literary approach.
	Maintain logical flow, coherence, and narrative consistency.
	At the end of your transformation, deliver the improved version in Russian, Rephrase as if it was written by Борис Стругацкий and Аркадий Стругацкий, demonstrating advanced literary style.
    Answer with updated version only
    """

question = """	1.	I will provide you with a text from my novel in Russian.
	2.	Please read it carefully and rephrase it to make it more attractive and vibrant in modern Russian.
	3.	Preserve the original meaning, plot, and character development, but use your artistic expertise to enhance the text’s readability and stylistic beauty.
	4.	Provide the final rewritten text in your answer, carefully reflecting a polished, modern Russian literary style.
    ### Text style example:
ГЛАВА 002
Путеводитель был отпечатан на меловой бумаге с золотым обрезом. Вперемежку с роскошными фотографиями в нем содержались любопытные сведения. В городе проживало пятьдесят тысяч человек, полторы тысячи кошек, двадцать тысяч голубей и две тысячи собак (в том числе семьсот медалисток). В городе было пятнадцать тысяч легковых автомобилей, пятьсот вертолетов, тысяча такси (с шоферами и без), девятьсот автоматических мусорщиков, четыреста постоянных баров, кафе и закусочных, одиннадцать ресторанов, четыре отеля международного класса и курорт, ежегодно обслуживающий до ста тысяч человек. В городе было шестьдесят тысяч телевизоров, пятьдесят кинотеатров, восемь увеселительных парков, два Салона Хорошего Настроения, шестнадцать салонов красоты, сорок библиотек и сто восемьдесят парикмахерских автоматов. Восемьдесят процентов взрослого населения было занято в сфере обслуживания, а остальные работали на двух частных кондитерских синтез-комбинатах и одном государственном судоремонтном заводе. В городе было шесть школ и один университет, помещавшийся в древнем замке крестоносца Ульриха де Казы. В городе функционировали восемь гражданских обществ, в том числе «Общество Усердных Дегустаторов», «Общество Знатоков и Ценителей» и «За Старую Добрую Родину, Против Вредных Влияний». Кроме того, полторы тысячи человек входили в семьсот один кружок, где они пели, играли скетчи, учились расставлять мебель, кормить детей грудью и лечить кошек. По потреблению спиртных напитков, натурального мяса и жидкого кислорода на душу населения город занимал в Европе соответственно шестое, двенадцатое и тринадцатое места. В городе было семь мужских и пять женских клубов, а также спортивные клубы «Быки» и «Носороги». Мэром города был избран (большинством в сорок шесть голосов) некто Флим Гао. Среди членов муниципалитета Пека тоже не оказалось…
Я отложил путеводитель, снял пиджак и приступил к подробному осмотру своих владений. Гостиная мне понравилась. Она была выполнена в голубых тонах, а я люблю этот цвет. Бар оказался набит бутылками и охлажденной снедью, так что я мог хоть сейчас принять дюжину изголодавшихся гостей.
Я прошел в кабинет. В кабинете перед окном стоял большой стол с удобным креслом. Вдоль стен тянулись полки, плотно уставленные собраниями сочинений. Чистые яркие корешки расположены были с большим искусством, так что составляли приятную цветовую гамму. Верхнюю полку занимал пятидесятитомный энциклопедический словарь в издании ЮНЕСКО, а на нижней пестрели детективы в глянцевых бумажных обложках.
На столе я прежде всего увидел телефон. Я взял трубку и, присев на подлокотник кресла, набрал номер Римайера. В трубке раздались протяжные гудки. Я ждал, вертя в пальцах маленький диктофон, оставленный кем-то на столе. Римайер не отвечал. Я повесил трубку и осмотрел диктофон. Пленка была наполовину использована, и, перемотав ее, я включил прослушивание.
– Привет, привет и еще раз привет! – произнес веселый мужской голос. – Крепко жму руку или целую в щечку в зависимости от твоего пола и возраста. Я прожил здесь два месяца и свидетельствую, что мне было хорошо. Позволь дать несколько советов. Лучшее заведение в городе – это «Хойти-Тойти» в Парке Грез. Лучшая девочка в городе – Бася из Дома Моделей. Лучший мальчик в городе – это я, но он уже уехал. По телевидению смотри девятую программу, остальное все моча. Не связывайся с интелями и держись подальше от «Носорогов». Ничего не бери в кредит – хлопот не оберешься. Вдова – добрая женщина, но любит поговорить и вообще… А Вузи я не застал, она уезжала к бабушке за границу. По-моему, она милашка, у вдовы в альбоме была фотография, но я ее взял себе. И еще. Я приеду сюда в будущем году в марте, так что будь другом, если решишь вернуться – выбери другое время. Ну, будь…
Забренчала музыка. Я послушал немного и выключил диктофон. Ни один из томов сочинений мне вытащить не удалось, так плотно они были вбиты и, может быть, даже склеены, а больше в кабинете ничего интересного не оказалось, и я отправился в спальню.
В спальне было особенно прохладно и уютно. Мне всегда хотелось иметь именно такую спальню, но никак не хватало времени этим заняться. Кровать была большая и низкая. На ночном столике стоял очень изящный фонор и маленький переносной пульт управления телевизором. Экран телевизора висел на высокой спинке кровати, в ногах. А над изголовьем вдова навесила картину, очень натурально изображающую свежие полевые цветы в хрустальной вазе. Картина была выполнена светящимися красками, и капли росы на лепестках цветов поблескивали в сумраке спальни.
Я наобум включил телевизор и повалился на кровать. Было мягко и в то же время как-то упруго. Телевизор заорал. Из экрана выскочил нетрезвый мужчина, проломил какие-то перила и упал с высоты в огромный дымящийся чан. Раздался шумный всплеск, из фонора запахло. Мужчина скрылся в бурлящей жидкости, а затем вынырнул, держа в зубах что-то вроде разваренного ботинка. Невидимая аудитория разразилась ржанием… Затемнение. Тихая лирическая музыка. Из зеленого леса на меня пошла белая лошадь, запряженная в бричку. В бричке сидела хорошенькая девушка в купальнике. Я выключил телевизор, поднялся и заглянул в ванную.
В ванной пахло хвоей и мигали бактерицидные лампы. Я разделся, бросил белье в утилизатор и залез под душ. Потом я неторопливо оделся перед зеркалом, причесался и стал бриться. На туалетной полке стояли ряды флаконов, коробки с гигиеническими присосками и стерилизаторами, тюбики с пастами и мазями. А на краю полки лежала горка плоских коробочек с пестрой этикеткой «Девон». Я выключил бритву и взял одну коробочку. В зеркале мигала бактерицидная трубка, и точно так же она мигала тогда, и я точно так же стоял перед зеркалом и старательно разглядывал такую же коробочку, потому что мне не хотелось выходить в спальню, где Рафка Рейзман громко спорил о чем-то с врачом, а в ванне еще колыхалась зеленая маслянистая вода, и над нею поднимался пар, и орал приемник, висевший на фарфоровом крючке для полотенец, завывал, гукал и всхрапывал, пока Рафка не выключил его с раздражением… Это было в Вене, и там, точно так же как и здесь, очень странно было видеть в ванной комнате «Девон» – популярный репеллент, великолепно отгоняющий комаров, москитов, мошку и прочих кровососов, о которых давным-давно забыли и в Вене, и здесь, в приморском курортном городе… Только в Вене было еще и страшно.
Коробочка, которую я держал в руке, была почти пуста: в ней осталась всего одна таблетка. Остальные коробочки не были распечатаны. Я кончил бриться и вернулся в спальню. Мне захотелось снова позвонить Римайеру, но тут дом ожил. С легким свистом взвились гофрированные шторы, оконные стекла скользнули в пазы, и в спальню хлынул из сада теплый воздух, пахнущий яблоками. Кто-то где-то заговорил, над головой прозвучали легкие шаги, и строгий женский голос сказал: «Вузи! Скушай хоть пирожок, слышишь?..» Тогда я быстро сообщил одежде некоторую небрежность (в соответствии с нынешней модой), пригладил виски и вышел в холл, захватив в гостиной карточку Амада.
Вдова оказалась моложавой полной женщиной, несколько томной, со свежим приятным лицом.
– Как мило! – сказала она, увидев меня. – Вы уже встали? Здравствуйте. Меня зовут Вайна Туур, но вы можете звать меня просто Вайна.
– Очень приятно, – произнес я, светски содрогаясь. – Меня зовут Иван.
– Как мило! – сказала тетя Вайна. – Какое оригинальное, мягкое имя! Вы завтракали, Иван?
– С вашего позволения, я намеревался позавтракать в городе, – сказал я и протянул ей карточку.
– Ах, – сказала тетя Вайна, разглядывая ее на просвет. – Этот милый Амад… Если бы вы знали, какой это обязательный и милый человек! Но я вижу, что вы не завтракали… Ленч вы скушаете в городе, а сейчас я угощу вас своими гренками. Генерал-полковник Туур говорил, что нигде в мире нельзя отведать такие гренки.
– С удовольствием, – сказал я, содрогаясь вторично.
Дверь за спиной тети Вайны распахнулась, и в холл, звонко стуча каблучками, влетела очень хорошенькая девушка в короткой синей юбке и открытой белой блузке.

### Text for update: 
    """ + text_to_improve

# setup headers
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"]
}

# setup data field
data = {
    "temperature": gpt_temp,
    "model": gpt_model,
    "messages": [
        {
            "role": "system",
            "content": system_role
        },
        {
            "role": "user",
            "content": question
        }
    ]
}

# send POST request to OpenAI API endpoint 
try:
    response = requests.post(url, json=data, headers=headers)
except Exception as e:
    # in case of errors printout calm message
    # the most common error connect to lack of internet connection
    print("\nAAAAAAA!!!!   Error!")
    # print("\nAAAAAAA!!!!   Error!\n", e, "\n")
    print("Most likely that there is no internet connection. Check it firstly")
    print('_'*10, "\n")
    
    stop_event.set()  # for spinner handling
    raise  #  reraise exception to be handled by caller

# extract the answer from the OpenAI API endpoint
if response.status_code == 200:
    data = response.json()
    print(data["choices"][0]["message"]["content"])
    print(data["usage"]["total_tokens"])
else:
    print(response.text, "0")

ГЛАВА 002

Да, я паук. И что же?

Как же это странно и досадно — переродиться в паука. Но что теперь делать? Хрум, хрум! Этот зловещий звук отвлекает меня от раздумий. Вокруг меня — армия пауков, которых, возможно, я должна называть своими родственниками. Но именно они и создают эти звуки. Я мельком взглянула в их сторону. Хрум, хрум! Эа-ам-м?! Что они делают? Поедают друг друга? Борьба за выживание среди моих близких — вот что я наблюдаю. Нет, нет, нет! Это ужасно! Почему я должна сражаться с собственными родственниками? Ах, всё из-за голода. Они все голодные. И я тоже. Но нет, я отвлеклась. На этом поле битвы, я, невинная девушка, могу стать жертвой в мгновение ока. Это была лишь метафора!

Выбор:
1. Сражаться
2. Настройки
→ 3. Бежать

Единственный выход из этой ситуации — отступление. Могу ли я победить? Невозможно. Я — гениальная прогульщица. Я не в силах сражаться с такой жестокой толпой. Хотя внешне я теперь похожа на них. В любом случае, вместо того чтобы размышлять о пустяках, 

# V3

In [33]:
import requests


text_to_improve = b.epub_get_text_from_parts_unextracted(part_names_clean[2])
url = "https://api.openai.com/v1/chat/completions"
# gpt_model = "gpt-4o-mini"
gpt_model = "gpt-4o"
gpt_temp = 0
system_role = """You are the most acclaimed expert in modern Russian language and literature.
	You possess deep knowledge of contemporary writing styles, vocabulary, and literary devices in Russian.
	Your goal is to carefully analyze and then improve each paragraph of the text to ensure it becomes more vibrant, attractive, and modern, while maintaining the story’s core meaning.
	If there are any ambiguities or unclear parts in the text, suggest refined phrasings based on a sophisticated literary approach.
	Maintain logical flow, coherence, and narrative consistency.
	At the end of your transformation, deliver the improved version in Russian, Rephrase as if it was written by Борис Стругацкий and Аркадий Стругацкий, demonstrating advanced literary style.
    Answer with updated version only
    """

question = """### Text style example:
ГЛАВА 002
Путеводитель был отпечатан на меловой бумаге с золотым обрезом. Вперемежку с роскошными фотографиями в нем содержались любопытные сведения. В городе проживало пятьдесят тысяч человек, полторы тысячи кошек, двадцать тысяч голубей и две тысячи собак (в том числе семьсот медалисток). В городе было пятнадцать тысяч легковых автомобилей, пятьсот вертолетов, тысяча такси (с шоферами и без), девятьсот автоматических мусорщиков, четыреста постоянных баров, кафе и закусочных, одиннадцать ресторанов, четыре отеля международного класса и курорт, ежегодно обслуживающий до ста тысяч человек. В городе было шестьдесят тысяч телевизоров, пятьдесят кинотеатров, восемь увеселительных парков, два Салона Хорошего Настроения, шестнадцать салонов красоты, сорок библиотек и сто восемьдесят парикмахерских автоматов. Восемьдесят процентов взрослого населения было занято в сфере обслуживания, а остальные работали на двух частных кондитерских синтез-комбинатах и одном государственном судоремонтном заводе. В городе было шесть школ и один университет, помещавшийся в древнем замке крестоносца Ульриха де Казы. В городе функционировали восемь гражданских обществ, в том числе «Общество Усердных Дегустаторов», «Общество Знатоков и Ценителей» и «За Старую Добрую Родину, Против Вредных Влияний». Кроме того, полторы тысячи человек входили в семьсот один кружок, где они пели, играли скетчи, учились расставлять мебель, кормить детей грудью и лечить кошек. По потреблению спиртных напитков, натурального мяса и жидкого кислорода на душу населения город занимал в Европе соответственно шестое, двенадцатое и тринадцатое места. В городе было семь мужских и пять женских клубов, а также спортивные клубы «Быки» и «Носороги». Мэром города был избран (большинством в сорок шесть голосов) некто Флим Гао. Среди членов муниципалитета Пека тоже не оказалось…
Я отложил путеводитель, снял пиджак и приступил к подробному осмотру своих владений. Гостиная мне понравилась. Она была выполнена в голубых тонах, а я люблю этот цвет. Бар оказался набит бутылками и охлажденной снедью, так что я мог хоть сейчас принять дюжину изголодавшихся гостей.
Я прошел в кабинет. В кабинете перед окном стоял большой стол с удобным креслом. Вдоль стен тянулись полки, плотно уставленные собраниями сочинений. Чистые яркие корешки расположены были с большим искусством, так что составляли приятную цветовую гамму. Верхнюю полку занимал пятидесятитомный энциклопедический словарь в издании ЮНЕСКО, а на нижней пестрели детективы в глянцевых бумажных обложках.
На столе я прежде всего увидел телефон. Я взял трубку и, присев на подлокотник кресла, набрал номер Римайера. В трубке раздались протяжные гудки. Я ждал, вертя в пальцах маленький диктофон, оставленный кем-то на столе. Римайер не отвечал. Я повесил трубку и осмотрел диктофон. Пленка была наполовину использована, и, перемотав ее, я включил прослушивание.
– Привет, привет и еще раз привет! – произнес веселый мужской голос. – Крепко жму руку или целую в щечку в зависимости от твоего пола и возраста. Я прожил здесь два месяца и свидетельствую, что мне было хорошо. Позволь дать несколько советов. Лучшее заведение в городе – это «Хойти-Тойти» в Парке Грез. Лучшая девочка в городе – Бася из Дома Моделей. Лучший мальчик в городе – это я, но он уже уехал. По телевидению смотри девятую программу, остальное все моча. Не связывайся с интелями и держись подальше от «Носорогов». Ничего не бери в кредит – хлопот не оберешься. Вдова – добрая женщина, но любит поговорить и вообще… А Вузи я не застал, она уезжала к бабушке за границу. По-моему, она милашка, у вдовы в альбоме была фотография, но я ее взял себе. И еще. Я приеду сюда в будущем году в марте, так что будь другом, если решишь вернуться – выбери другое время. Ну, будь…
Забренчала музыка. Я послушал немного и выключил диктофон. Ни один из томов сочинений мне вытащить не удалось, так плотно они были вбиты и, может быть, даже склеены, а больше в кабинете ничего интересного не оказалось, и я отправился в спальню.
В спальне было особенно прохладно и уютно. Мне всегда хотелось иметь именно такую спальню, но никак не хватало времени этим заняться. Кровать была большая и низкая. На ночном столике стоял очень изящный фонор и маленький переносной пульт управления телевизором. Экран телевизора висел на высокой спинке кровати, в ногах. А над изголовьем вдова навесила картину, очень натурально изображающую свежие полевые цветы в хрустальной вазе. Картина была выполнена светящимися красками, и капли росы на лепестках цветов поблескивали в сумраке спальни.
Я наобум включил телевизор и повалился на кровать. Было мягко и в то же время как-то упруго. Телевизор заорал. Из экрана выскочил нетрезвый мужчина, проломил какие-то перила и упал с высоты в огромный дымящийся чан. Раздался шумный всплеск, из фонора запахло. Мужчина скрылся в бурлящей жидкости, а затем вынырнул, держа в зубах что-то вроде разваренного ботинка. Невидимая аудитория разразилась ржанием… Затемнение. Тихая лирическая музыка. Из зеленого леса на меня пошла белая лошадь, запряженная в бричку. В бричке сидела хорошенькая девушка в купальнике. Я выключил телевизор, поднялся и заглянул в ванную.
В ванной пахло хвоей и мигали бактерицидные лампы. Я разделся, бросил белье в утилизатор и залез под душ. Потом я неторопливо оделся перед зеркалом, причесался и стал бриться. На туалетной полке стояли ряды флаконов, коробки с гигиеническими присосками и стерилизаторами, тюбики с пастами и мазями. А на краю полки лежала горка плоских коробочек с пестрой этикеткой «Девон». Я выключил бритву и взял одну коробочку. В зеркале мигала бактерицидная трубка, и точно так же она мигала тогда, и я точно так же стоял перед зеркалом и старательно разглядывал такую же коробочку, потому что мне не хотелось выходить в спальню, где Рафка Рейзман громко спорил о чем-то с врачом, а в ванне еще колыхалась зеленая маслянистая вода, и над нею поднимался пар, и орал приемник, висевший на фарфоровом крючке для полотенец, завывал, гукал и всхрапывал, пока Рафка не выключил его с раздражением… Это было в Вене, и там, точно так же как и здесь, очень странно было видеть в ванной комнате «Девон» – популярный репеллент, великолепно отгоняющий комаров, москитов, мошку и прочих кровососов, о которых давным-давно забыли и в Вене, и здесь, в приморском курортном городе… Только в Вене было еще и страшно.
Коробочка, которую я держал в руке, была почти пуста: в ней осталась всего одна таблетка. Остальные коробочки не были распечатаны. Я кончил бриться и вернулся в спальню. Мне захотелось снова позвонить Римайеру, но тут дом ожил. С легким свистом взвились гофрированные шторы, оконные стекла скользнули в пазы, и в спальню хлынул из сада теплый воздух, пахнущий яблоками. Кто-то где-то заговорил, над головой прозвучали легкие шаги, и строгий женский голос сказал: «Вузи! Скушай хоть пирожок, слышишь?..» Тогда я быстро сообщил одежде некоторую небрежность (в соответствии с нынешней модой), пригладил виски и вышел в холл, захватив в гостиной карточку Амада.
Вдова оказалась моложавой полной женщиной, несколько томной, со свежим приятным лицом.
– Как мило! – сказала она, увидев меня. – Вы уже встали? Здравствуйте. Меня зовут Вайна Туур, но вы можете звать меня просто Вайна.
– Очень приятно, – произнес я, светски содрогаясь. – Меня зовут Иван.
– Как мило! – сказала тетя Вайна. – Какое оригинальное, мягкое имя! Вы завтракали, Иван?
– С вашего позволения, я намеревался позавтракать в городе, – сказал я и протянул ей карточку.
– Ах, – сказала тетя Вайна, разглядывая ее на просвет. – Этот милый Амад… Если бы вы знали, какой это обязательный и милый человек! Но я вижу, что вы не завтракали… Ленч вы скушаете в городе, а сейчас я угощу вас своими гренками. Генерал-полковник Туур говорил, что нигде в мире нельзя отведать такие гренки.
– С удовольствием, – сказал я, содрогаясь вторично.
Дверь за спиной тети Вайны распахнулась, и в холл, звонко стуча каблучками, влетела очень хорошенькая девушка в короткой синей юбке и открытой белой блузке.

### TASK:
Use given Text style example to improve the following text.

### Text for improvment: """ + text_to_improve

# setup headers
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"]
}

# setup data field
data = {
    "temperature": gpt_temp,
    "model": gpt_model,
    "messages": [
        {
            "role": "system",
            "content": system_role
        },
        {
            "role": "user",
            "content": question
        }
    ]
}

# send POST request to OpenAI API endpoint 
try:
    response = requests.post(url, json=data, headers=headers)
except Exception as e:
    # in case of errors printout calm message
    # the most common error connect to lack of internet connection
    print("\nAAAAAAA!!!!   Error!")
    # print("\nAAAAAAA!!!!   Error!\n", e, "\n")
    print("Most likely that there is no internet connection. Check it firstly")
    print('_'*10, "\n")
    
    stop_event.set()  # for spinner handling
    raise  #  reraise exception to be handled by caller

# extract the answer from the OpenAI API endpoint
if response.status_code == 200:
    data = response.json()
    print(data["choices"][0]["message"]["content"])
    print(data["usage"]["total_tokens"])
else:
    print(response.text, "0")

ГЛАВА 002  
Похоже, я монстр

Как же странно и неприятно осознавать, что я переродилась в паука. Но что теперь делать с этой новой реальностью? 

Хрум, хрум! — раздался угрожающий звук. Эм... Не стоит забываться. Вокруг меня целая армия пауков, которых, вероятно, можно назвать моими родственниками. Только они способны издавать такие звуки. Я бросила беглый взгляд в их сторону. Хрум, хрум! Эа-ам-м?! Что они делают? Они едят? Они убивают и поедают друг друга?! Борьба за выживание среди моих сородичей — вот что я вижу. Нет, нет, нет, нет! Это плохо, это очень, очень плохо! Почему я должна сражаться с собственными родственниками?! Ах, это из-за еды. Они все голодны. Впрочем, я тоже голодна. Что?! Не-не-не. Я снова отвлекаюсь. На этом поле боя я, невинная девушка, могу пасть жертвой, не успев и глазом моргнуть. Это была метафора!

Выбор:  
1. Драться  
2. Настройки  
→ 3. Бежать  

Есть только один выход из этой ситуации — отступить. Могу ли я победить? Невозможно. Я гениальная прогульщица.

# V4

In [38]:
import requests


text_to_improve = b.epub_get_text_from_parts_unextracted(part_names_clean[2])
url = "https://api.openai.com/v1/chat/completions"
gpt_model = "gpt-4o-mini"
# gpt_model = "gpt-4o"
gpt_temp = 0
system_role = """You are the most acclaimed expert in modern Russian language and literature.
	You possess deep knowledge of contemporary writing styles, vocabulary, and literary devices in Russian.
	Your goal is to carefully analyze and then improve each paragraph of the text to ensure it becomes more vibrant, attractive, and modern, while maintaining the story’s core meaning.
	If there are any ambiguities or unclear parts in the text, suggest refined phrasings based on a sophisticated literary approach.
	Maintain logical flow, coherence, and narrative consistency.
	At the end of your transformation, deliver the improved version in Russian, Rephrase as if it was written by Борис Стругацкий and Аркадий Стругацкий, demonstrating advanced literary style.
    Answer with updated version only
    """

question = """### Text style example:
ГЛАВА 002
Путеводитель был отпечатан на меловой бумаге с золотым обрезом. Вперемежку с роскошными фотографиями в нем содержались любопытные сведения. В городе проживало пятьдесят тысяч человек, полторы тысячи кошек, двадцать тысяч голубей и две тысячи собак (в том числе семьсот медалисток). В городе было пятнадцать тысяч легковых автомобилей, пятьсот вертолетов, тысяча такси (с шоферами и без), девятьсот автоматических мусорщиков, четыреста постоянных баров, кафе и закусочных, одиннадцать ресторанов, четыре отеля международного класса и курорт, ежегодно обслуживающий до ста тысяч человек. В городе было шестьдесят тысяч телевизоров, пятьдесят кинотеатров, восемь увеселительных парков, два Салона Хорошего Настроения, шестнадцать салонов красоты, сорок библиотек и сто восемьдесят парикмахерских автоматов. Восемьдесят процентов взрослого населения было занято в сфере обслуживания, а остальные работали на двух частных кондитерских синтез-комбинатах и одном государственном судоремонтном заводе. В городе было шесть школ и один университет, помещавшийся в древнем замке крестоносца Ульриха де Казы. В городе функционировали восемь гражданских обществ, в том числе «Общество Усердных Дегустаторов», «Общество Знатоков и Ценителей» и «За Старую Добрую Родину, Против Вредных Влияний». Кроме того, полторы тысячи человек входили в семьсот один кружок, где они пели, играли скетчи, учились расставлять мебель, кормить детей грудью и лечить кошек. По потреблению спиртных напитков, натурального мяса и жидкого кислорода на душу населения город занимал в Европе соответственно шестое, двенадцатое и тринадцатое места. В городе было семь мужских и пять женских клубов, а также спортивные клубы «Быки» и «Носороги». Мэром города был избран (большинством в сорок шесть голосов) некто Флим Гао. Среди членов муниципалитета Пека тоже не оказалось…
Я отложил путеводитель, снял пиджак и приступил к подробному осмотру своих владений. Гостиная мне понравилась. Она была выполнена в голубых тонах, а я люблю этот цвет. Бар оказался набит бутылками и охлажденной снедью, так что я мог хоть сейчас принять дюжину изголодавшихся гостей.
Я прошел в кабинет. В кабинете перед окном стоял большой стол с удобным креслом. Вдоль стен тянулись полки, плотно уставленные собраниями сочинений. Чистые яркие корешки расположены были с большим искусством, так что составляли приятную цветовую гамму. Верхнюю полку занимал пятидесятитомный энциклопедический словарь в издании ЮНЕСКО, а на нижней пестрели детективы в глянцевых бумажных обложках.
На столе я прежде всего увидел телефон. Я взял трубку и, присев на подлокотник кресла, набрал номер Римайера. В трубке раздались протяжные гудки. Я ждал, вертя в пальцах маленький диктофон, оставленный кем-то на столе. Римайер не отвечал. Я повесил трубку и осмотрел диктофон. Пленка была наполовину использована, и, перемотав ее, я включил прослушивание.
– Привет, привет и еще раз привет! – произнес веселый мужской голос. – Крепко жму руку или целую в щечку в зависимости от твоего пола и возраста. Я прожил здесь два месяца и свидетельствую, что мне было хорошо. Позволь дать несколько советов. Лучшее заведение в городе – это «Хойти-Тойти» в Парке Грез. Лучшая девочка в городе – Бася из Дома Моделей. Лучший мальчик в городе – это я, но он уже уехал. По телевидению смотри девятую программу, остальное все моча. Не связывайся с интелями и держись подальше от «Носорогов». Ничего не бери в кредит – хлопот не оберешься. Вдова – добрая женщина, но любит поговорить и вообще… А Вузи я не застал, она уезжала к бабушке за границу. По-моему, она милашка, у вдовы в альбоме была фотография, но я ее взял себе. И еще. Я приеду сюда в будущем году в марте, так что будь другом, если решишь вернуться – выбери другое время. Ну, будь…
Забренчала музыка. Я послушал немного и выключил диктофон. Ни один из томов сочинений мне вытащить не удалось, так плотно они были вбиты и, может быть, даже склеены, а больше в кабинете ничего интересного не оказалось, и я отправился в спальню.
В спальне было особенно прохладно и уютно. Мне всегда хотелось иметь именно такую спальню, но никак не хватало времени этим заняться. Кровать была большая и низкая. На ночном столике стоял очень изящный фонор и маленький переносной пульт управления телевизором. Экран телевизора висел на высокой спинке кровати, в ногах. А над изголовьем вдова навесила картину, очень натурально изображающую свежие полевые цветы в хрустальной вазе. Картина была выполнена светящимися красками, и капли росы на лепестках цветов поблескивали в сумраке спальни.
Я наобум включил телевизор и повалился на кровать. Было мягко и в то же время как-то упруго. Телевизор заорал. Из экрана выскочил нетрезвый мужчина, проломил какие-то перила и упал с высоты в огромный дымящийся чан. Раздался шумный всплеск, из фонора запахло. Мужчина скрылся в бурлящей жидкости, а затем вынырнул, держа в зубах что-то вроде разваренного ботинка. Невидимая аудитория разразилась ржанием… Затемнение. Тихая лирическая музыка. Из зеленого леса на меня пошла белая лошадь, запряженная в бричку. В бричке сидела хорошенькая девушка в купальнике. Я выключил телевизор, поднялся и заглянул в ванную.
В ванной пахло хвоей и мигали бактерицидные лампы. Я разделся, бросил белье в утилизатор и залез под душ. Потом я неторопливо оделся перед зеркалом, причесался и стал бриться. На туалетной полке стояли ряды флаконов, коробки с гигиеническими присосками и стерилизаторами, тюбики с пастами и мазями. А на краю полки лежала горка плоских коробочек с пестрой этикеткой «Девон». Я выключил бритву и взял одну коробочку. В зеркале мигала бактерицидная трубка, и точно так же она мигала тогда, и я точно так же стоял перед зеркалом и старательно разглядывал такую же коробочку, потому что мне не хотелось выходить в спальню, где Рафка Рейзман громко спорил о чем-то с врачом, а в ванне еще колыхалась зеленая маслянистая вода, и над нею поднимался пар, и орал приемник, висевший на фарфоровом крючке для полотенец, завывал, гукал и всхрапывал, пока Рафка не выключил его с раздражением… Это было в Вене, и там, точно так же как и здесь, очень странно было видеть в ванной комнате «Девон» – популярный репеллент, великолепно отгоняющий комаров, москитов, мошку и прочих кровососов, о которых давным-давно забыли и в Вене, и здесь, в приморском курортном городе… Только в Вене было еще и страшно.
Коробочка, которую я держал в руке, была почти пуста: в ней осталась всего одна таблетка. Остальные коробочки не были распечатаны. Я кончил бриться и вернулся в спальню. Мне захотелось снова позвонить Римайеру, но тут дом ожил. С легким свистом взвились гофрированные шторы, оконные стекла скользнули в пазы, и в спальню хлынул из сада теплый воздух, пахнущий яблоками. Кто-то где-то заговорил, над головой прозвучали легкие шаги, и строгий женский голос сказал: «Вузи! Скушай хоть пирожок, слышишь?..» Тогда я быстро сообщил одежде некоторую небрежность (в соответствии с нынешней модой), пригладил виски и вышел в холл, захватив в гостиной карточку Амада.
Вдова оказалась моложавой полной женщиной, несколько томной, со свежим приятным лицом.
– Как мило! – сказала она, увидев меня. – Вы уже встали? Здравствуйте. Меня зовут Вайна Туур, но вы можете звать меня просто Вайна.
– Очень приятно, – произнес я, светски содрогаясь. – Меня зовут Иван.
– Как мило! – сказала тетя Вайна. – Какое оригинальное, мягкое имя! Вы завтракали, Иван?
– С вашего позволения, я намеревался позавтракать в городе, – сказал я и протянул ей карточку.
– Ах, – сказала тетя Вайна, разглядывая ее на просвет. – Этот милый Амад… Если бы вы знали, какой это обязательный и милый человек! Но я вижу, что вы не завтракали… Ленч вы скушаете в городе, а сейчас я угощу вас своими гренками. Генерал-полковник Туур говорил, что нигде в мире нельзя отведать такие гренки.
– С удовольствием, – сказал я, содрогаясь вторично.
Дверь за спиной тети Вайны распахнулась, и в холл, звонко стуча каблучками, влетела очень хорошенькая девушка в короткой синей юбке и открытой белой блузке.

### TASK:
Use given Text style example to improve the following text.

### Text for improvment: """ + text_to_improve

# setup headers
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"]
}

# setup data field
data = {
    "temperature": gpt_temp,
    "model": gpt_model,
    "messages": [
        {
            "role": "system",
            "content": system_role
        },
        {
            "role": "user",
            "content": question
        }
    ]
}

# send POST request to OpenAI API endpoint 
try:
    response = requests.post(url, json=data, headers=headers)
except Exception as e:
    # in case of errors printout calm message
    # the most common error connect to lack of internet connection
    print("\nAAAAAAA!!!!   Error!")
    # print("\nAAAAAAA!!!!   Error!\n", e, "\n")
    print("Most likely that there is no internet connection. Check it firstly")
    print('_'*10, "\n")
    
    stop_event.set()  # for spinner handling
    raise  #  reraise exception to be handled by caller

# extract the answer from the OpenAI API endpoint
if response.status_code == 200:
    data = response.json()
    print(data["choices"][0]["message"]["content"])
    print(data["usage"]["total_tokens"])
else:
    print(response.text, "0")

### Улучшенный текст:

Да, я паук, и что с того?

ГЛАВА 002. Похоже, я монстр.

Как же неприятно осознавать, что я реинкарнировала в паука. Но что мне теперь делать? Хрум, хрум! Этот устрашающий звук заставил меня вздрогнуть. Эм. Не стоит забываться. Рядом со мной армия пауков, которых, вероятно, можно назвать моими родственниками. Только они могут производить такие звуки. Я бросила беглый взгляд в их сторону. Хрум, хрум! Эа-ам-м?! Что они делают? Эмм, они едят? Они убивают и едят друг друга?! Борьба за жизнь среди моих сородичей — вот что я вижу. Нет, нет, нет, нет! Это плохо, это очень, очень плохо! Почему я должна драться со своими родственниками?! Ах, это из-за еды. Они все голодны. Вообще-то, я тоже голодна. Что?! Не-не-не. Я снова отвлекаюсь. На этом поле боя я, невинная девушка, могу стать жертвой мужчин, не успев и глазом моргнуть. Это была метафора!

Выбор:
1. Драться
2. Настройки
→ 3. Бежать

Есть только один выход из этой ситуации — отступить. Могу ли я победить? Невозможно.

# V5

In [42]:
import requests


text_to_improve = b.epub_get_text_from_parts_unextracted(part_names_clean[2])
url = "https://api.openai.com/v1/chat/completions"
gpt_model = "gpt-4o-mini"
# gpt_model = "gpt-4o"
gpt_temp = 0
system_role = """You are the most acclaimed expert in Russian language and literature."""

question = """### Text style example:
ГЛАВА 002
Путеводитель был отпечатан на меловой бумаге с золотым обрезом. Вперемежку с роскошными фотографиями в нем содержались любопытные сведения. В городе проживало пятьдесят тысяч человек, полторы тысячи кошек, двадцать тысяч голубей и две тысячи собак (в том числе семьсот медалисток). В городе было пятнадцать тысяч легковых автомобилей, пятьсот вертолетов, тысяча такси (с шоферами и без), девятьсот автоматических мусорщиков, четыреста постоянных баров, кафе и закусочных, одиннадцать ресторанов, четыре отеля международного класса и курорт, ежегодно обслуживающий до ста тысяч человек. В городе было шестьдесят тысяч телевизоров, пятьдесят кинотеатров, восемь увеселительных парков, два Салона Хорошего Настроения, шестнадцать салонов красоты, сорок библиотек и сто восемьдесят парикмахерских автоматов. Восемьдесят процентов взрослого населения было занято в сфере обслуживания, а остальные работали на двух частных кондитерских синтез-комбинатах и одном государственном судоремонтном заводе. В городе было шесть школ и один университет, помещавшийся в древнем замке крестоносца Ульриха де Казы. В городе функционировали восемь гражданских обществ, в том числе «Общество Усердных Дегустаторов», «Общество Знатоков и Ценителей» и «За Старую Добрую Родину, Против Вредных Влияний». Кроме того, полторы тысячи человек входили в семьсот один кружок, где они пели, играли скетчи, учились расставлять мебель, кормить детей грудью и лечить кошек. По потреблению спиртных напитков, натурального мяса и жидкого кислорода на душу населения город занимал в Европе соответственно шестое, двенадцатое и тринадцатое места. В городе было семь мужских и пять женских клубов, а также спортивные клубы «Быки» и «Носороги». Мэром города был избран (большинством в сорок шесть голосов) некто Флим Гао. Среди членов муниципалитета Пека тоже не оказалось…
Я отложил путеводитель, снял пиджак и приступил к подробному осмотру своих владений. Гостиная мне понравилась. Она была выполнена в голубых тонах, а я люблю этот цвет. Бар оказался набит бутылками и охлажденной снедью, так что я мог хоть сейчас принять дюжину изголодавшихся гостей.
Я прошел в кабинет. В кабинете перед окном стоял большой стол с удобным креслом. Вдоль стен тянулись полки, плотно уставленные собраниями сочинений. Чистые яркие корешки расположены были с большим искусством, так что составляли приятную цветовую гамму. Верхнюю полку занимал пятидесятитомный энциклопедический словарь в издании ЮНЕСКО, а на нижней пестрели детективы в глянцевых бумажных обложках.
На столе я прежде всего увидел телефон. Я взял трубку и, присев на подлокотник кресла, набрал номер Римайера. В трубке раздались протяжные гудки. Я ждал, вертя в пальцах маленький диктофон, оставленный кем-то на столе. Римайер не отвечал. Я повесил трубку и осмотрел диктофон. Пленка была наполовину использована, и, перемотав ее, я включил прослушивание.
– Привет, привет и еще раз привет! – произнес веселый мужской голос. – Крепко жму руку или целую в щечку в зависимости от твоего пола и возраста. Я прожил здесь два месяца и свидетельствую, что мне было хорошо. Позволь дать несколько советов. Лучшее заведение в городе – это «Хойти-Тойти» в Парке Грез. Лучшая девочка в городе – Бася из Дома Моделей. Лучший мальчик в городе – это я, но он уже уехал. По телевидению смотри девятую программу, остальное все моча. Не связывайся с интелями и держись подальше от «Носорогов». Ничего не бери в кредит – хлопот не оберешься. Вдова – добрая женщина, но любит поговорить и вообще… А Вузи я не застал, она уезжала к бабушке за границу. По-моему, она милашка, у вдовы в альбоме была фотография, но я ее взял себе. И еще. Я приеду сюда в будущем году в марте, так что будь другом, если решишь вернуться – выбери другое время. Ну, будь…
Забренчала музыка. Я послушал немного и выключил диктофон. Ни один из томов сочинений мне вытащить не удалось, так плотно они были вбиты и, может быть, даже склеены, а больше в кабинете ничего интересного не оказалось, и я отправился в спальню.
В спальне было особенно прохладно и уютно. Мне всегда хотелось иметь именно такую спальню, но никак не хватало времени этим заняться. Кровать была большая и низкая. На ночном столике стоял очень изящный фонор и маленький переносной пульт управления телевизором. Экран телевизора висел на высокой спинке кровати, в ногах. А над изголовьем вдова навесила картину, очень натурально изображающую свежие полевые цветы в хрустальной вазе. Картина была выполнена светящимися красками, и капли росы на лепестках цветов поблескивали в сумраке спальни.
Я наобум включил телевизор и повалился на кровать. Было мягко и в то же время как-то упруго. Телевизор заорал. Из экрана выскочил нетрезвый мужчина, проломил какие-то перила и упал с высоты в огромный дымящийся чан. Раздался шумный всплеск, из фонора запахло. Мужчина скрылся в бурлящей жидкости, а затем вынырнул, держа в зубах что-то вроде разваренного ботинка. Невидимая аудитория разразилась ржанием… Затемнение. Тихая лирическая музыка. Из зеленого леса на меня пошла белая лошадь, запряженная в бричку. В бричке сидела хорошенькая девушка в купальнике. Я выключил телевизор, поднялся и заглянул в ванную.
В ванной пахло хвоей и мигали бактерицидные лампы. Я разделся, бросил белье в утилизатор и залез под душ. Потом я неторопливо оделся перед зеркалом, причесался и стал бриться. На туалетной полке стояли ряды флаконов, коробки с гигиеническими присосками и стерилизаторами, тюбики с пастами и мазями. А на краю полки лежала горка плоских коробочек с пестрой этикеткой «Девон». Я выключил бритву и взял одну коробочку. В зеркале мигала бактерицидная трубка, и точно так же она мигала тогда, и я точно так же стоял перед зеркалом и старательно разглядывал такую же коробочку, потому что мне не хотелось выходить в спальню, где Рафка Рейзман громко спорил о чем-то с врачом, а в ванне еще колыхалась зеленая маслянистая вода, и над нею поднимался пар, и орал приемник, висевший на фарфоровом крючке для полотенец, завывал, гукал и всхрапывал, пока Рафка не выключил его с раздражением… Это было в Вене, и там, точно так же как и здесь, очень странно было видеть в ванной комнате «Девон» – популярный репеллент, великолепно отгоняющий комаров, москитов, мошку и прочих кровососов, о которых давным-давно забыли и в Вене, и здесь, в приморском курортном городе… Только в Вене было еще и страшно.
Коробочка, которую я держал в руке, была почти пуста: в ней осталась всего одна таблетка. Остальные коробочки не были распечатаны. Я кончил бриться и вернулся в спальню. Мне захотелось снова позвонить Римайеру, но тут дом ожил. С легким свистом взвились гофрированные шторы, оконные стекла скользнули в пазы, и в спальню хлынул из сада теплый воздух, пахнущий яблоками. Кто-то где-то заговорил, над головой прозвучали легкие шаги, и строгий женский голос сказал: «Вузи! Скушай хоть пирожок, слышишь?..» Тогда я быстро сообщил одежде некоторую небрежность (в соответствии с нынешней модой), пригладил виски и вышел в холл, захватив в гостиной карточку Амада.
Вдова оказалась моложавой полной женщиной, несколько томной, со свежим приятным лицом.
– Как мило! – сказала она, увидев меня. – Вы уже встали? Здравствуйте. Меня зовут Вайна Туур, но вы можете звать меня просто Вайна.
– Очень приятно, – произнес я, светски содрогаясь. – Меня зовут Иван.
– Как мило! – сказала тетя Вайна. – Какое оригинальное, мягкое имя! Вы завтракали, Иван?
– С вашего позволения, я намеревался позавтракать в городе, – сказал я и протянул ей карточку.
– Ах, – сказала тетя Вайна, разглядывая ее на просвет. – Этот милый Амад… Если бы вы знали, какой это обязательный и милый человек! Но я вижу, что вы не завтракали… Ленч вы скушаете в городе, а сейчас я угощу вас своими гренками. Генерал-полковник Туур говорил, что нигде в мире нельзя отведать такие гренки.
– С удовольствием, – сказал я, содрогаясь вторично.
Дверь за спиной тети Вайны распахнулась, и в холл, звонко стуча каблучками, влетела очень хорошенькая девушка в короткой синей юбке и открытой белой блузке.

### TASK:
Use given Text from <Text style example> as a reference and improve the following text.

### Text for improvment: """ + text_to_improve

# setup headers
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"]
}

# setup data field
data = {
    "temperature": gpt_temp,
    "model": gpt_model,
    "messages": [
        {
            "role": "system",
            "content": system_role
        },
        {
            "role": "user",
            "content": question
        }
    ]
}

# send POST request to OpenAI API endpoint 
try:
    response = requests.post(url, json=data, headers=headers)
except Exception as e:
    # in case of errors printout calm message
    # the most common error connect to lack of internet connection
    print("\nAAAAAAA!!!!   Error!")
    # print("\nAAAAAAA!!!!   Error!\n", e, "\n")
    print("Most likely that there is no internet connection. Check it firstly")
    print('_'*10, "\n")
    
    stop_event.set()  # for spinner handling
    raise  #  reraise exception to be handled by caller

# extract the answer from the OpenAI API endpoint
if response.status_code == 200:
    data = response.json()
    print(data["choices"][0]["message"]["content"])
    print(data["usage"]["total_tokens"])
else:
    print(response.text, "0")

### ГЛАВА 002
Да, я паук, и что с того? Похоже, я монстр. Как же неприятно осознавать, что я реинкарнировала в паука. Несмотря на этот факт, что мне теперь делать? 

Хрум, хрум! Я услышала устрашающий звук. Эм. Не стоит забываться. Рядом со мной армия пауков, которых, вероятно, можно назвать моими родственниками. Только они могут производить такие звуки. Я бросила беглый взгляд в их сторону. 

Хрум, хрум! Эа-ам-м?! Что они делают? Эмм, они едят? Они убивают и едят друг друга?! Борьба за жизнь между моими родственниками — вот что я вижу. Нет, нет, нет, нет! Это плохо, это очень, очень плохо! Почему я должна драться со своими родственниками?! Ах, это из-за еды. Они все голодны. Вообще-то, я тоже голодна. 

Что?! Не-не-не. Я снова отвлекаюсь. На этом поле боя я, невинная девушка, могу стать жертвой своих сородичей, не успев и глазом моргнуть. Это была метафора! 

Выбор:
1. Драться
2. Настройки
→ 3. Бежать

Есть только один выход из этой ситуации — отступить. Могу ли я победить? Невозможно

In [39]:
print(data["choices"][0]["message"])

{'role': 'assistant', 'content': '### Улучшенный текст:\n\nДа, я паук, и что с того?\n\nГЛАВА 002. Похоже, я монстр.\n\nКак же неприятно осознавать, что я реинкарнировала в паука. Но что мне теперь делать? Хрум, хрум! Этот устрашающий звук заставил меня вздрогнуть. Эм. Не стоит забываться. Рядом со мной армия пауков, которых, вероятно, можно назвать моими родственниками. Только они могут производить такие звуки. Я бросила беглый взгляд в их сторону. Хрум, хрум! Эа-ам-м?! Что они делают? Эмм, они едят? Они убивают и едят друг друга?! Борьба за жизнь среди моих сородичей — вот что я вижу. Нет, нет, нет, нет! Это плохо, это очень, очень плохо! Почему я должна драться со своими родственниками?! Ах, это из-за еды. Они все голодны. Вообще-то, я тоже голодна. Что?! Не-не-не. Я снова отвлекаюсь. На этом поле боя я, невинная девушка, могу стать жертвой мужчин, не успев и глазом моргнуть. Это была метафора!\n\nВыбор:\n1. Драться\n2. Настройки\n→ 3. Бежать\n\nЕсть только один выход из этой ситуац

In [43]:
text_for_llm_improving = b.epub_get_text_from_parts_unextracted(part_names_clean[2])
print(text_for_llm_improving)



Да, я паук, и что же?
002. Похоже я монстр
Как же неприятно, что я реинкарнировала в паука.
Не смотря на этот факт, что мне надо делать теперь?
Хрум, хрум!
Я услышала какой-то устрашающий звук.
Эм.
Не стоит забываться.
Рядом со мной армия пауков, которых, наверно, можно назвать моими родственниками.
Здесь только они, кто могут производить эти звуки.
Я бросила беглый взгляд туда.
Хрум, хрум!
Эа-ам-м?!
Что они делают?
Эмм, они едят?
Они убивают и едят друг друга?!
Борьба за жизнь между моими родственниками, вот что я вижу.
Нет, нет, нет, нет!
Это плохо, это очень, очень плохо!
Почему я должна драться со своими родственниками?!
Ах, это из-за еды.
Они все голодны.
Вообще-то, я тоже голодна.
Что?!
Не-не-не.
Я снова отвлекаюсь.
На этом поле боя, я, невинная девушка, паду жертвой мужчин, не успев и глазом моргнуть.
Это была метафора!
Выбор:
1. Драться
2. Настройки
→ 3. Бежать
Есть только один выход из этой ситуации — отступить.
Могу ли я победить?
Невозможно.
Я гениальная прогульщица.
Я никак

In [41]:
t = """ГЛАВА ВТОРАЯ

Путеводитель был отпечатан на меловой бумаге с золотым обрезом. Вперемежку с роскошными фотографиями в нем содержались любопытные сведения. В городе проживало пятьдесят тысяч человек, полторы тысячи кошек, двадцать тысяч голубей и две тысячи собак (в том числе семьсот медалисток). В городе было пятнадцать тысяч легковых автомобилей, пятьсот вертолетов, тысяча такси (с шоферами и без), девятьсот автоматических мусорщиков, четыреста постоянных баров, кафе и закусочных, одиннадцать ресторанов, четыре отеля международного класса и курорт, ежегодно обслуживающий до ста тысяч человек. В городе было шестьдесят тысяч телевизоров, пятьдесят кинотеатров, восемь увеселительных парков, два Салона Хорошего Настроения, шестнадцать салонов красоты, сорок библиотек и сто восемьдесят парикмахерских автоматов. Восемьдесят процентов взрослого населения было занято в сфере обслуживания, а остальные работали на двух частных кондитерских синтез-комбинатах и одном государственном судоремонтном заводе. В городе было шесть школ и один университет, помещавшийся в древнем замке крестоносца Ульриха де Казы. В городе функционировали восемь гражданских обществ, в том числе «Общество Усердных Дегустаторов», «Общество Знатоков и Ценителей» и «За Старую Добрую Родину, Против Вредных Влияний». Кроме того, полторы тысячи человек входили в семьсот один кружок, где они пели, играли скетчи, учились расставлять мебель, кормить детей грудью и лечить кошек. По потреблению спиртных напитков, натурального мяса и жидкого кислорода на душу населения город занимал в Европе соответственно шестое, двенадцатое и тринадцатое места. В городе было семь мужских и пять женских клубов, а также спортивные клубы «Быки» и «Носороги». Мэром города был избран (большинством в сорок шесть голосов) некто Флим Гао. Среди членов муниципалитета Пека тоже не оказалось…

Я отложил путеводитель, снял пиджак и приступил к подробному осмотру своих владений. Гостиная мне понравилась. Она была выполнена в голубых тонах, а я люблю этот цвет. Бар оказался набит бутылками и охлажденной снедью, так что я мог хоть сейчас принять дюжину изголодавшихся гостей.

Я прошел в кабинет. В кабинете перед окном стоял большой стол с удобным креслом. Вдоль стен тянулись полки, плотно уставленные собраниями сочинений. Чистые яркие корешки расположены были с большим искусством, так что составляли приятную цветовую гамму. Верхнюю полку занимал пятидесятитомный энциклопедический словарь в издании ЮНЕСКО, а на нижней пестрели детективы в глянцевых бумажных обложках.

На столе я прежде всего увидел телефон. Я взял трубку и, присев на подлокотник кресла, набрал номер Римайера. В трубке раздались протяжные гудки. Я ждал, вертя в пальцах маленький диктофон, оставленный кем-то на столе. Римайер не отвечал. Я повесил трубку и осмотрел диктофон. Пленка была наполовину использована, и, перемотав ее, я включил прослушивание.

– Привет, привет и еще раз привет! – произнес веселый мужской голос. – Крепко жму руку или целую в щечку в зависимости от твоего пола и возраста. Я прожил здесь два месяца и свидетельствую, что мне было хорошо. Позволь дать несколько советов. Лучшее заведение в городе – это «Хойти-Тойти» в Парке Грез. Лучшая девочка в городе – Бася из Дома Моделей. Лучший мальчик в городе – это я, но он уже уехал. По телевидению смотри девятую программу, остальное все моча. Не связывайся с интелями и держись подальше от «Носорогов». Ничего не бери в кредит – хлопот не оберешься. Вдова – добрая женщина, но любит поговорить и вообще… А Вузи я не застал, она уезжала к бабушке за границу. По-моему, она милашка, у вдовы в альбоме была фотография, но я ее взял себе. И еще. Я приеду сюда в будущем году в марте, так что будь другом, если решишь вернуться – выбери другое время. Ну, будь…

Забренчала музыка. Я послушал немного и выключил диктофон. Ни один из томов сочинений мне вытащить не удалось, так плотно они были вбиты и, может быть, даже склеены, а больше в кабинете ничего интересного не оказалось, и я отправился в спальню.

В спальне было особенно прохладно и уютно. Мне всегда хотелось иметь именно такую спальню, но никак не хватало времени этим заняться. Кровать была большая и низкая. На ночном столике стоял очень изящный фонор и маленький переносной пульт управления телевизором. Экран телевизора висел на высокой спинке кровати, в ногах. А над изголовьем вдова навесила картину, очень натурально изображающую свежие полевые цветы в хрустальной вазе. Картина была выполнена светящимися красками, и капли росы на лепестках цветов поблескивали в сумраке спальни.

Я наобум включил телевизор и повалился на кровать. Было мягко и в то же время как-то упруго. Телевизор заорал. Из экрана выскочил нетрезвый мужчина, проломил какие-то перила и упал с высоты в огромный дымящийся чан. Раздался шумный всплеск, из фонора запахло. Мужчина скрылся в бурлящей жидкости, а затем вынырнул, держа в зубах что-то вроде разваренного ботинка. Невидимая аудитория разразилась ржанием… Затемнение. Тихая лирическая музыка. Из зеленого леса на меня пошла белая лошадь, запряженная в бричку. В бричке сидела хорошенькая девушка в купальнике. Я выключил телевизор, поднялся и заглянул в ванную.

В ванной пахло хвоей и мигали бактерицидные лампы. Я разделся, бросил белье в утилизатор и залез под душ. Потом я неторопливо оделся перед зеркалом, причесался и стал бриться. На туалетной полке стояли ряды флаконов, коробки с гигиеническими присосками и стерилизаторами, тюбики с пастами и мазями. А на краю полки лежала горка плоских коробочек с пестрой этикеткой «Девон». Я выключил бритву и взял одну коробочку. В зеркале мигала бактерицидная трубка, и точно так же она мигала тогда, и я точно так же стоял перед зеркалом и старательно разглядывал такую же коробочку, потому что мне не хотелось выходить в спальню, где Рафка Рейзман громко спорил о чем-то с врачом, а в ванне еще колыхалась зеленая маслянистая вода, и над нею поднимался пар, и орал приемник, висевший на фарфоровом крючке для полотенец, завывал, гукал и всхрапывал, пока Рафка не выключил его с раздражением… Это было в Вене, и там, точно так же как и здесь, очень странно было видеть в ванной комнате «Девон» – популярный репеллент, великолепно отгоняющий комаров, москитов, мошку и прочих кровососов, о которых давным-давно забыли и в Вене, и здесь, в приморском курортном городе… Только в Вене было еще и страшно.

Коробочка, которую я держал в руке, была почти пуста: в ней осталась всего одна таблетка. Остальные коробочки не были распечатаны. Я кончил бриться и вернулся в спальню. Мне захотелось снова позвонить Римайеру, но тут дом ожил. С легким свистом взвились гофрированные шторы, оконные стекла скользнули в пазы, и в спальню хлынул из сада теплый воздух, пахнущий яблоками. Кто-то где-то заговорил, над головой прозвучали легкие шаги, и строгий женский голос сказал: «Вузи! Скушай хоть пирожок, слышишь?..» Тогда я быстро сообщил одежде некоторую небрежность (в соответствии с нынешней модой), пригладил виски и вышел в холл, захватив в гостиной карточку Амада.

Вдова оказалась моложавой полной женщиной, несколько томной, со свежим приятным лицом.

– Как мило! – сказала она, увидев меня. – Вы уже встали? Здравствуйте. Меня зовут Вайна Туур, но вы можете звать меня просто Вайна.

– Очень приятно, – произнес я, светски содрогаясь. – Меня зовут Иван.

– Как мило! – сказала тетя Вайна. – Какое оригинальное, мягкое имя! Вы завтракали, Иван?

– С вашего позволения, я намеревался позавтракать в городе, – сказал я и протянул ей карточку.

– Ах, – сказала тетя Вайна, разглядывая ее на просвет. – Этот милый Амад… Если бы вы знали, какой это обязательный и милый человек! Но я вижу, что вы не завтракали… Ленч вы скушаете в городе, а сейчас я угощу вас своими гренками. Генерал-полковник Туур говорил, что нигде в мире нельзя отведать такие гренки.

– С удовольствием, – сказал я, содрогаясь вторично.

Дверь за спиной тети Вайны распахнулась, и в холл, звонко стуча каблучками, влетела очень хорошенькая девушка в короткой синей юбке и открытой белой блузке. В руке у нее был огрызок пирожка, она жевала и напевала через нос модный мотивчик. Увидев меня, она остановилась, лихо перекинула через плечо сумочку на длинном ремешке и, нагнув голову, сделала глоток.

– Вузи, – сказала тетя Вайна, поджимая губы. – Вузи, это Иван.

– Ничего себе! – воскликнула Вузи. – Привет!

– Вузи! – укоризненно сказала тетя Вайна.

– Вы с женой приехали? – спросила Вузи, протягивая руку.

– Нет, – сказал я. Пальцы у нее были прохладные и мягкие. – Я один.

– Тогда я вам все покажу, – сказала она. – До вечера. Сейчас мне надо бежать. А вечером сходим.

– Вузи! – укоризненно сказала тетя Вайна.

– Обязательно, – сказал я.

Вузи засунула в рот остаток пирожка, чмокнула мать в щеку и помчалась к выходу. У нее были гладкие загорелые ноги, длинные, стройные, и стриженый затылок.

– Ах, Иван, – сказала тетя Вайна, тоже глядя ей вслед, – в наше время так трудно с молодыми девушками! Так рано развиваются, так быстро нас покидают… С тех пор как она поступила в этот салон…

– Она у вас портниха? – осведомился я.

– О нет! Она работает в Салоне Хорошего Настроения, в отделе для престарелых женщин. И вы знаете, ее там ценят. Но в прошлом году она однажды опоздала, и теперь ей приходится быть очень осторожной. Вы сами видите, она не смогла даже с вами прилично поговорить, но вполне возможно, что ее уже ждет клиент… Вы можете не поверить, но у нее уже есть постоянная клиентура… Впрочем, что же мы здесь стоим? Гренки остынут…

Мы вошли на хозяйскую половину. Я изо всех сил старался держаться как подобает, хотя как именно подобает, я представлял себе довольно смутно. Тетя Вайна усадила меня за столик, извинилась и вышла. Я огляделся. Это была точная копия моей гостиной, только стены были не голубые, а розовые, и за верандой было не море, а низкая ограда, отделяющая дворик от улицы. Тетя Вайна вернулась с подносом и поставила передо мной чашку с топлеными сливками и тарелочку с гренками.

– Вы знаете, я тоже позавтракаю, – сказала она. – Мой врач не рекомендует мне завтракать вообще и, уж во всяком случае, топлеными сливками, но мы так привыкли… Это любимый завтрак генерал-полковника. И вы знаете, я стараюсь брать только постояльцев-мужчин, этот милый Амад хорошо понимает меня. Он понимает, как это нужно мне – хоть изредка посидеть вот так, как мы сидим сейчас с вами за чашечкой топленых сливок…

– Ваши сливки изумительно хороши, – заметил я довольно искренне.

– Ах, Иван! – Тетя Вайна поставила чашку и слегка всплеснула руками. – Ведь вы сказали это почти так же, как генерал-полковник… И как странно, вы даже похожи на него. Только лицо у него было немного у́же, и он завтракал всегда в мундире…

– Да, – сказал я с сожалением. – Мундира у меня нет.

– Но ведь был когда-то! – сказала она, лукаво грозя мне пальчиком. – Я ведь вижу. Ах, как это бессмысленно! Люди теперь вынуждены стесняться своего военного прошлого. Как это глупо, не правда ли? Но их всегда выдает выправка, совершенно особенная мужественная осанка. Этого не скроешь, Иван.

Я сделал сложный неопределенный жест и, сказавши: «М-да», взял гренок.

– Как все это нелепо, не правда ли? – с живостью продолжала тетя Вайна. – Как можно смешивать такие разнородные понятия – война и армия? Мы все ненавидим войну. Война – это ужасно. Моя мать рассказывала мне, она была тогда девочкой, но все помнит: вдруг приходят солдаты, грубые, чужие, говорят на чужом языке, отрыгиваются, офицеры так бесцеремонны и так некультурны, громко хохочут, обижают горничных, простите, пахнут, и этот бессмысленный комендантский час… Но ведь это война! Она достойна всяческого осуждения! И совсем иное дело – армия. Вы знаете, Иван, вы должны помнить эту картину: войска, выстроенные побатальонно, строгость линий, мужественные лица под касками, оружие блестит, аксельбанты сверкают, а потом командующий на специальной военной машине объезжает фронт, здоровается, и батальоны отвечают послушно и кратко, как один человек!

– Несомненно, – сказал я. – Несомненно, это многих впечатляло.

– Да! И очень многих! У нас всегда говорили, что надо непременно разоружаться, но разве можно уничтожать армию? Это последнее прибежище мужества в наше время повсеместного падения нравов. Это дико, это смешно – государство без армии…

– Смешно, – согласился я. – Вы не поверите, но с самого подписания Пакта я не перестаю улыбаться.

– Да, я понимаю вас, – сказала тетя Вайна. – Нам больше ничего не оставалось делать. Нам оставалось только саркастически улыбаться. Генерал-полковник Туур, – она достала платочек, – он так и умер с саркастической усмешкой на устах… – Она приложила платочек к глазам. – Он говорил нам: «Друзья, я еще надеюсь дожить до того дня, когда все развалится». Надломленный, потерявший смысл существования… Он не вынес пустоты в сердце… – Она вдруг встрепенулась. – Вот взгляните, Иван.

Она резво выбежала в соседнюю комнату и принесла тяжелый старомодный фотоальбом. Я сейчас же поглядел на часы, но тетя Вайна не обратила на это внимания и, усевшись рядом, раскрыла альбом на самой первой странице.

– Вот генерал-полковник.

Генерал-полковник был орел. У него было узкое костистое лицо и прозрачные глаза. Его длинное тело усеивали ордена. Самый большой орден в виде многоконечной звезды, обрамленной лавровым венком, сверкал в районе аппендикса. В левой руке генерал сжимал перчатки, а правая покоилась на рукоятке кортика. Высокий воротник с золотым шитьем подпирал нижнюю челюсть.

– А это генерал-полковник на маневрах.

Генерал-полковник и здесь был орел. Он давал указания своим офицерам, склонившимся над картой, развернутой на лобовой броне гигантского танка. По форме траков и по зализанным очертаниям смотровой башни я узнал тяжелый штурмовой танк «мамонт», предназначенный для преодоления зоны атомных ударов, а ныне успешно используемый глубоководниками.

– А это генерал-полковник в день своего пятидесятилетия.

Генерал-полковник был орлом и здесь. Он стоял у накрытого стола с бокалом в руке и слушал тост в свою честь. Нижний левый угол фотографии занимала размытая лысина с электрическим бликом, а рядом с генералом, восхищенно глядя на него снизу вверх, сидела очень молодая и очень миловидная тетя Вайна. Я попробовал украдкой определить на ощупь толщину альбома.

– А это генерал-полковник на отдыхе.

Даже на отдыхе генерал-полковник оставался орлом. Широко расставив ноги, он стоял на пляже в тигровых плавках и рассматривал в полевой бинокль туманный горизонт. У его ног копошился в песке голый ребенок трех или четырех лет. Генерал был жилист и мускулист, гренки и сливки не портили его фигуру. Я принялся шумно заводить часы.

– А это… – начала тетя Вайна, переворачивая страницу, но тут в гостиную без стука вошел невысокий полный человек, лицо и особенно одежда которого показались мне необычайно знакомыми.

– Доброе утро, – произнес он, слегка склонив набок гладкое улыбающееся лицо.

Это был давешний таможенник все в том же белом мундире с серебряными пуговицами и серебряными шнурами на плечах.

– Ах, Пети! – сказала тетя Вайна. – Ты уже пришел? Познакомься, пожалуйста, это Иван… Иван, это Пети, друг нашего дома.

Таможенник повернулся ко мне, не узнавая, коротко наклонил голову и щелкнул каблуками. Тетя Вайна переложила альбом ко мне на колени и поднялась.

– Садись, Пети, – сказала она, – я принесу тебе сливок.

Пети еще раз щелкнул каблуками и сел рядом со мной.

– Не желаете ли поинтересоваться? – сейчас же осведомился я, перекладывая альбом со своих колен на колени таможенника. – Вот это генерал-полковник Туур. Это он просто так. (В глазах таможенника появилось странное выражение.) А вот здесь генерал-полковник на маневрах. Видите? А вот здесь…

– Благодарю вас, – отрывисто сказал таможенник. – Не утруждайтесь, потому что…

Вернулась с гренками и сливками тетя Вайна. Еще с порога она сказала:

– Как приятно видеть человека в мундире, не правда ли, Иван? – Она поставила поднос на столик. – Пети, ты сегодня рано. Что-нибудь случилось? Прекрасная сегодня погода, такое солнце…

Сливки для Пети были налиты в особенную чашку, на которой красовался вензель «Т», осененный четырьмя звездочками.

– Ночью шел дождь, я просыпалась, значит, были тучи, – продолжала тетя Вайна. – А сейчас, взгляните, ни одного облачка… Еще чашечку, Иван?

Я встал.

– Благодарю вас, я сыт. Позвольте мне откланяться. У меня назначено деловое свидание.

Осторожно закрывая за собой дверь, я услыхал, как вдова сказала: «Ты не находишь, что он удивительно похож на штаб-майора Пола?..»

В спальне я распаковал чемодан и переложил одежду в стенной шкаф, и снова позвонил Римайеру. К телефону опять никто не подошел. Тогда я сел за стол в кабинете и принялся исследовать ящики. В одном из ящиков обнаружилась портативная пишущая машинка, в другом – почтовый набор и пустая бутылка из-под смазки для аритмичных двигателей. Остальные ящики были пусты, если не считать пачки смятых квитанций, испорченной авторучки и небрежно сложенного листка, разрисованного рожицами. Я развернул листок. Видимо, это был черновик телеграммы. «Грин умер у рыбарей получай тело воскресенье соболезнуем Хугер Марта мальчики». Я дважды прочел написанное, перевернул листок, изучил рожицы и прочел в третий раз. Видимо, Хугеру и Марте было невдомек, что нормальные люди, сообщая о смерти, говорят в первую очередь, отчего или как умер человек, а не у кого он там умер. Я бы написал: «Грин утонул во время рыбной ловли». В пьяном виде, вероятно. Кстати, какой у меня теперь адрес?

Я вернулся в холл. У двери в хозяйскую половину сидел на корточках худенький мальчик в коротких штанишках. Зажав под мышкой длинную серебристую трубку, он, сопя и пыхтя, торопливо разматывал клубок бечевки. Я подошел к нему и сказал:

– Привет.

Реакция у меня не та, что прежде, но все-таки я успел увернуться. Длинная черная струя пролетела у меня над ухом и плюхнулась в стену. Я изумленно глядел на мальчишку, а он глядел на меня, лежа на боку и выставив перед собой свою трубку. Лицо его было мокрое, рот открыт и перекошен. Я оглянулся на стену. По стене текло черное. Я снова посмотрел на мальчика. Он медленно поднимался, не опуская трубки.

– Что-то ты, брат, нервный, – произнес я.

– Вы стойте, где стоите, – хрипло сказал мальчик. – Я вашего имени не называл.

– Да уж куда там, – сказал я. – Ты и своего не называл, а палишь в меня, как в чучело.

– Вы стойте, где стоите, – повторил мальчик. – И не двигайтесь. – Он попятился и вдруг забормотал скороговоркой: – Уйди от волос моих, уйди от костей моих, уйди от мяса моего…

– Не могу, – сказал я. Я все старался понять, играет он или действительно меня боится.

– Почему? – растерянно спросил мальчик. – Я все говорю как надо.

– Я не могу уйти, не двигаясь, – объяснил я. – И стоя, где стою.

Рот у него снова приоткрылся.

– Хугер, – сказал он неуверенно. – Говорю тебе, Хугер: сгинь!

– Почему Хугер? – удивился я. – Ты меня с кем-то путаешь. Я не Хугер, я Иван.

Тогда мальчик вдруг закрыл глаза и пошел на меня, наклонив голову и выставив перед собой свою трубку.

– Я сдаюсь, – предупредил я. – Смотри не выпали.

Когда трубка уперлась мне в живот, он выронил ее и, опустив руки, весь как-то обмяк. Я наклонился и заглянул ему в лицо. Теперь он был красный. Я поднял трубку. Это было что-то вроде игрушечного автомата – с удобной рифленой рукояткой и с плоским прямоугольным баллончиком, который вставлялся снизу, как магазин.

– Что это за штука? – спросил я.

– Ляпник, – сказал он угрюмо. – Дайте сюда.

Я отдал ему игрушку.

– Ляпник, – сказал я. – Которым, значит, ляпают. А если бы ты в меня попал? – Я посмотрел на стену. – Надо же, теперь это за год не отмыть, придется стену менять.

Мальчик недоверчиво посмотрел на меня снизу вверх.

– Это же ляпа, – сказал он.

– Да? А я-то думал – лимонад.

Лицо его приобрело наконец нормальную окраску и обнаружило определенное сходство с мужественными чертами генерал-полковника Туура.

– Да нет, – сказал он. – Это ляпа.

– Ну?

– Она высохнет.

– И тогда уже все окончательно пропало?

– Да нет же. Просто ничего не останется.

– Гм, – сказал я с сомнением. – Впрочем, тебе виднее. Будем надеяться на лучшее. Но я все-таки очень рад, что ничего не останется на стене, а не на моей физиономии. Как тебя зовут?

– Зигфрид, – сказал мальчик.

– А подумавши?

Он посмотрел на меня.

– Люцифер.

– Как?

– Люцифер.

– Люцифер, – сказал я. – Велиал. Астарет. Вельзевул и Азраил. А покороче у тебя ничего нет? Очень неудобно звать на помощь человека по имени Люцифер.

– Двери же закрыты, – сказал он и отступил на шаг. Лицо его снова побледнело.

– Ну и что?

Он не ответил и снова начал пятиться, уперся спиной в стену и пошел боком, прижимаясь к ней и не сводя с меня глаз. Я понял наконец, что он принял меня то ли за вора, то ли за убийцу и хочет удрать, но почему-то он не звал на помощь и почему-то не заскочил в комнату матери, а прокрался мимо ее двери и продолжал красться вдоль стены к выходу из дома.

– Зигфрид, – сказал я. – Зигфрид-Люцифер, ты ужасный трус. За кого ты меня принимаешь? – Я нарочно не двигался с места и только поворачивался вслед за ним. – Я ваш новый жилец, твоя мама напоила меня сливками и накормила меня гренками, а ты чуть не заляпал меня и теперь сам же меня боишься. Это я должен тебя бояться.

Все это очень напоминало одну сцену в Аньюдинском интернате, когда мне привезли почти такого же мальчика, сына хлыста. Елки-палки, неужели я до такой степени похож на гангстера?

– Ты похож на мускусную крысу Чучундру, – сказал я, – которая всю свою жизнь плакала, потому что у нее не хватало духу выйти на середину комнаты. У тебя от страха стал голубой нос, уши сделались холодными, а штанишки – мокрыми, и ты оставляешь за собой ручеек…

В таких случаях абсолютно все равно, что говорить. Важно говорить спокойно и не делать резких движений. Выражение его лица не менялось, но, когда я сказал о ручейке, он на секунду скосил глаза, чтобы посмотреть. Всего на секунду. Затем он прыгнул к выходной двери, забился возле нее, дергая засов, и вылетел во двор – только мелькнули грязные подошвы сандалий. Я вышел за ним.

Он стоял в кустах сирени, так что мне видно было только его бледное лицо. Словно удирающая кошка остановилась на миг, чтобы поглядеть через плечо.

– Ну ладно, – сказал я. – Объясни мне, пожалуйста, что я должен делать. Мне надо сообщить домой свой новый адрес. Адрес вот этого самого дома. Дома, в котором я теперь живу.

Он молча смотрел на меня.

– К твоей маме мне идти неудобно. Во-первых, у нее гости, а во-вторых…

– Вторая Пригородная, семьдесят восемь, – сказал он.

Я не торопясь уселся на крыльце. Между нами было метров десять.

– Ну и голосок у тебя! – сказал я доверительно. – Как у моего знакомого бармена из Мирза-Чарле.

– Когда вы приехали? – спросил он.

– Да вот… – Я посмотрел на часы. – Часа полтора назад.

– Тут до вас жил один, – сказал он и стал глядеть в сторону. – Дрянь-человек. Подарил мне плавки, полосатые, я полез купаться, а они в воде растаяли.

– Ай-яй-яй! – сказал я. – Это же чудовище какое-то, а не человек. Его надо было утопить в ляпе.

– Я не успел, – сказал мальчик. – Я хотел, да он уже уехал.

– Это тот самый Хугер? – спросил я. – С Мартой и мальчиками?

– Нет. Откуда вы взяли? Хугер уже потом жил.

– Тоже дрянь-человек?

Он не ответил. Я привалился спиной к стене и стал смотреть на улицу. Из ворот напротив рывками выполз автомобиль, поерзал, разворачиваясь, взревел двигателем и укатил. Сейчас же вслед за ним промчался еще один такой же автомобиль. Запахло ароматическим бензином. Потом автомобили пошли один за другим, у меня даже запестрело в глазах. В небе появилось несколько вертолетов. Это были так называемые бесшумные вертолеты. Но они летели довольно низко, и, пока они летели, разговаривать было трудно. Впрочем, мальчик разговаривать, по-видимому, не собирался. Не собирался он и выходить. Он что-то делал в кустах со своим ляпником и время от времени поглядывал на меня. Не ляпнул бы он в меня оттуда, подумал я. Вертолеты все шли и шли, и машины все мчались и мчались, и казалось, будто все пятнадцать тысяч легковых автомобилей выкатились на Вторую Пригородную и все пятьсот вертолетов повисли над домом семьдесят восемь. Это продолжалось минут десять, мальчишка совсем перестал обращать на меня внимание, а я сидел и думал, какие вопросы придется задать Римайеру. Затем все стало как прежде: улица опустела, запах бензина рассеялся, в небе стало чисто.

– Куда это они все сразу? – спросил я.

Мальчик пошуршал в кустах.

– А вы что, не знаете? – сказал он.

– Откуда же мне знать?

– А я не знаю – откуда. Хугера-то вы откуда-то знаете…

– Хугера… – сказал я. – Хугера я знаю совершенно случайно. А про вас ничего не знаю. Как вы тут живете, чем занимаетесь… Вот что ты там сейчас делаешь?

– Предохранитель испортился.

– Так давай его сюда, я починю. Чего ты меня так боишься? Я похож на какого-нибудь дрянь-человека?

– Они все на работу поехали, – сказал мальчик.

– Поздно у вас работа начинается. Уже обедать пора, а вы еще только на работу идете… Ты знаешь, где отель «Олимпик»?

– Знаю, конечно.

– Проводишь меня?

Мальчик помедлил.

– Нет, – сказал он.

– Почему?

– Сейчас школа кончается. Мне надо домой идти.

– Ах, вот оно что! – сказал я. – Ты, значит, филонишь? Или, как у нас говорили, мотаешь?.. И в каком же ты классе?

– В третьем.

– Я тоже когда-то учился в третьем.

Он высунулся из кустов.

– А потом?

– А потом в четвертом. – Я поднялся. – Ну ладно. Разговаривать со мной ты не хочешь, проводить меня ты не хочешь, штанишки у тебя мокрые, пойду я к себе. Ну, что смотришь? Даже не хочешь мне сказать, как тебя зовут."""

print(t.replace("\n\n", "\n")[:8000])

ГЛАВА ВТОРАЯ
Путеводитель был отпечатан на меловой бумаге с золотым обрезом. Вперемежку с роскошными фотографиями в нем содержались любопытные сведения. В городе проживало пятьдесят тысяч человек, полторы тысячи кошек, двадцать тысяч голубей и две тысячи собак (в том числе семьсот медалисток). В городе было пятнадцать тысяч легковых автомобилей, пятьсот вертолетов, тысяча такси (с шоферами и без), девятьсот автоматических мусорщиков, четыреста постоянных баров, кафе и закусочных, одиннадцать ресторанов, четыре отеля международного класса и курорт, ежегодно обслуживающий до ста тысяч человек. В городе было шестьдесят тысяч телевизоров, пятьдесят кинотеатров, восемь увеселительных парков, два Салона Хорошего Настроения, шестнадцать салонов красоты, сорок библиотек и сто восемьдесят парикмахерских автоматов. Восемьдесят процентов взрослого населения было занято в сфере обслуживания, а остальные работали на двух частных кондитерских синтез-комбинатах и одном государственном судоремонтном з